In [ ]:
# Data Flattening
import numpy as np
import pandas as pd

import datetime
from datetime import datetime

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, MapType, LongType
from pyspark.sql.functions import lit, udf, size, avg, min as min_, max as max_, sum as sum_, count, countDistinct, col, mean, stddev, struct, explode, explode_outer, unix_timestamp, sum as sum_
from operator import itemgetter
import json

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import Bucketizer

from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import StandardScaler

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, row_number, dense_rank, lead, lag, rank
from pyspark.sql.window import Window

from pyspark.ml.classification import LogisticRegressionModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import collections
from collections import deque
import redis
import json


In [ ]:
'''
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats/oc-stats-staging/dt=2019-09-01/hr=01/*')
rdd = df.rdd
r = rdd.map(lambda x: [(x['id'], x['when'], t['name'], t['rx_bytes'],t['tx_bytes'],t['rx_bps'],t['tx_bps'],t['rx_packets'],t['tx_packets']) for t in x['device']['interfaces']])
r1 = r.flatMap(lambda x: [t for t in x])
df1 = sqlContext.createDataFrame(r1)
df1 = df1.withColumnRenamed('_1','id')
tsudf = udf(lambda x: x[0]*1000000000+x[1], LongType())
df1 = df1.withColumn('ts', tsudf(struct(df1['_2']['seconds'],df1['_2']['nanos'])))
df1 = df1.withColumnRenamed('_3','name') \
        .withColumnRenamed('_4','rx_bytes') \
        .withColumnRenamed('_5','tx_bytes') \
        .withColumnRenamed('_6','rx_bps') \
        .withColumnRenamed('_7','tx_bps') \
        .withColumnRenamed('_8','rx_pkts') \
        .withColumnRenamed('_9','tx_pkts') \
        .drop('_2')

id_name_gp = df1.groupby('id','name').agg(sum_('rx_bytes'),sum_('tx_bytes'),sum_('rx_bps'),sum_('tx_bps'),sum_('rx_pkts'),sum_('tx_pkts'))
id_gp = df1.groupby('id').agg(sum_('rx_bytes'),sum_('tx_bytes'),sum_('rx_bps'),sum_('tx_bps'),sum_('rx_pkts'),sum_('tx_pkts'))
'''

In [2]:
#flatten interface
date = '2019-10-25'
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats-analytics/oc-stats-analytics-staging/dt='+date+'/*')

dfin = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                 explode_outer('interfaces').alias('interfaces'))

dfin = dfin.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                  col('interfaces.name').alias('name'),
                   col('interfaces.link').alias('link'),
                   col('interfaces.full_duplex').alias('full_duplex'),
                   col('interfaces.mbps').alias('mbps'),
                   col('interfaces.mtu').alias('mtu'),
                   col('interfaces.address').alias('address'),
                   col('interfaces.admin_status').alias('admin_status'),
                   col('interfaces.last_flapped').alias('last_flapped'),
                   col('interfaces.errors').alias('errors'),
                   col('interfaces.poe_enabled').alias('poe_enabled'),
                   col('interfaces.poe_status').alias('poe_status'),
                   col('interfaces.poe_power_limit').alias('poe_power_limit'),
                   col('interfaces.poe_power').alias('poe_power'),
                   col('interfaces.poe_priority').alias('poe_priority'),
                   col('interfaces.poe_class').alias('poe_class'),
                   col('interfaces.poe_mode').alias('poe_mode'),
                   col('interfaces.rx_bytes').alias('rx_bytes'),
                   col('interfaces.tx_bytes').alias('tx_bytes'),
                   col('interfaces.rx_packets').alias('rx_packets'),
                   col('interfaces.tx_packets').alias('tx_packets'),
                   col('interfaces.rx_bps').alias('rx_bps'),
                   col('interfaces.tx_bps').alias('tx_bps'),
                   col('interfaces.rx_errors').alias('rx_errors'),
                   col('interfaces.rx_undersize_errors').alias('rx_undersize_errors'),
                   col('interfaces.rx_oversize_errors').alias('rx_oversize_errors'),
                   col('interfaces.rx_fcserrors').alias('rx_fcserrors'),
                   col('interfaces.rx_overrun_errors').alias('rx_overrun_errors'),
                   col('interfaces.rx_discards').alias('rx_discards'),
                   col('interfaces.tx_errors').alias('tx_errors'),
                   col('interfaces.tx_drops').alias('tx_drops'),
                   col('interfaces.tx_mtuerrors').alias('tx_mtuerrors'),
                   col('interfaces.txcarrier_transition').alias('txcarrier_transition'),
                   col('interfaces.tx_mcast_packets').alias('tx_mcast_packets'),
                   col('interfaces.tx_bcast_packets').alias('tx_bcast_packets'),
                   col('interfaces.rx_mcast_packets').alias('rx_mcast_packets'),
                   col('interfaces.rx_bcast_packets').alias('rx_bcast_packets'),
                   col('interfaces.rx_l3_incompletes').alias('rx_l3_incompletes'),
                   col('interfaces.rx_l2_channel_error').alias('rx_l2_channel_error'),
                   col('interfaces.rx_l2_mismatch_timeouts').alias('rx_l2_mismatch_timeouts'),
                   col('interfaces.rx_fifo_errors').alias('rx_fifo_errors'),
                   col('interfaces.rx_resource_errors').alias('rx_resource_errors'),
                   col('interfaces.auto_negotiation_status').alias('auto_negotiation_status'),
                   col('interfaces.media_type').alias('media_type'))

vp_udf = udf(lambda x: 'phy' if (x[:2]=='ge') | (x[:2]=='xe') | (x[:2]=='et') else 'vir', StringType())
dfin = dfin.withColumn('type', vp_udf('name'))

dfin = dfin.filter(col('media_type')=='copper')

convert_time = udf(lambda x: datetime.utcfromtimestamp(x/1000000).strftime('%Y-%m-%dT%H:%M:%S.%f+00:00')[:16], StringType())
dfin = dfin.withColumn('time', convert_time('when'))

pct = udf(lambda x: x[0]/x[1], FloatType())
dfin = dfin.withColumn('power_pct', pct(struct('poe_power','poe_power_limit')))

dfin = dfin.filter(col('delta')==True)

reix = udf(lambda x: int(x[3]), IntegerType())
dfin = dfin.withColumn('re_ix', reix(dfin.name))

dfin.persist().first()
dfin.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/interfaces'+date)
dfin.unpersist()

SyntaxError: invalid syntax (<ipython-input-2-ca1f9c7e8200>, line 104)

In [ ]:
dfeng = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                 explode_outer('chassis_routing_engines').alias('eng'))

dfeng = dfeng.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                  col('eng.module_index').alias('module_index'),
                     col('eng.is_master').alias('is_master'),
                     col('eng.status').alias('status'),
                     col('eng.temp_celsius').alias('temp_celsius'),
                     col('eng.memory_total_mb').alias('memory_total_mb'),
                     col('eng.memory_in_use_mb').alias('memory_in_use_mb'),
                     col('eng.cpu_user').alias('cpu_user'),
                     col('eng.cpu_background').alias('cpu_background'),
                     col('eng.cpu_system').alias('cpu_system'),
                     col('eng.cpu_interrupt').alias('cpu_interrupt'),
                     col('eng.cpu_idle').alias('cpu_idle'),
                     col('eng.cputemp_celsius').alias('cputemp_celsius'),
                     col('eng.model').alias('eng_model'),
                     col('eng.last_reboot_reason').alias('last_reboot_reason'),
                     col('eng.load_avg').alias('load_avg'))
                     
dfeng = dfeng.withColumn('load_avg1', dfeng.load_avg[0])  
memorypct = udf(lambda x: x[0]/x[1], FloatType())
dfeng = dfeng.withColumn('memory_pct', memorypct(struct('memory_in_use_mb','memory_total_mb')))
dfeng = dfeng.filter(col('delta')==True)


In [ ]:
+-----+-----------------------+------+                                          
| link|auto_negotiation_status| count|
+-----+-----------------------+------+
|false|             complete  | 10625|
| true|             complete  | 35540|
| true|         not applicable| 69870|
|false|         not applicable| 17192|
|false|     no-autonegotiation|     5|
|false|           incomplete  |300389|
|false|                       |  1752|
| true|           incomplete  |     1|
| true|                       |  3066|
+-----+-----------------------+------+

In [ ]:
dfeng.groupby('id').agg(min_('loadavg1'),max_('loadavg1'),avg('loadavg1'),stddev('loadavg1'), \
                    min_('temp_celcius'),max_('temp_celcius'),avg('temp_celcius'),stddev('temp_celcius'), \
                    min_('eng_cputempc'),max_('eng_cputempc'),avg('eng_cputempc'),stddev('eng_cputempc'), \
                    min_('temp_tempc'),max_('temp_tempc'),avg('temp_tempc'),stddev('temp_tempc'), \
                    min_('eng_memoryinusemb'),max_('eng_memoryinusemb'),avg('eng_memoryinusemb'),stddev('eng_memoryinusemb'), \
                    min_('eng_cpuuser'),max_('eng_cpuuser'),avg('eng_cpuuser'),stddev('eng_cpuuser'), \
                    min_('eng_cpubackground'),max_('eng_cpubackground'),avg('eng_cpubackground'),stddev('eng_cpubackground'), \
                    min_('eng_cpusystem'),max_('eng_cpusystem'),avg('eng_cpusystem'),stddev('eng_cpusystem'), \
                    min_('eng_cpuinterrupt'),max_('eng_cpuinterrupt'),avg('eng_cpuinterrupt'),stddev('eng_cpuinterrupt'), \
                    min_('eng_cpuidle'),max_('eng_cpuidle'),avg('eng_cpuidle'),stddev('eng_cpuidle'))

In [3]:
import pandas as pd
data = pd.read_csv('~/Downloads/switchhealth.csv')

In [4]:
data.describe()


,Unnamed: 0,when,temp_celsius,memory_total_mb,memory_in_use_mb,cpu_user,cpu_background,cpu_system,cpu_interrupt,cpu_idle,cputemp_celsius,load_avg1,memory_pct
count,14639.00000,1.463900e+04,14639.000000,14639.000000,14639.000000,14639.000000,14639.0,14639.000000,14639.000000,14639.000000,14639.000000,14639.000000,14639.000000
mean,7319.00000,1.571832e+15,50.789535,1975.583100,492.963454,20.175900,0.0,27.841792,2.623950,49.327686,50.789535,0.869112,0.245799
std,4226.05963,2.524428e+10,11.594184,46.828187,348.796338,11.806798,0.0,17.602726,2.476407,30.720582,11.594184,0.628476,0.167351
min,0.00000,1.571789e+15,29.000000,1945.000000,233.000000,1.000000,0.0,1.000000,0.000000,0.000000,29.000000,0.000000,0.119733
25%,3659.50000,1.571810e+15,45.000000,1945.000000,272.000000,7.000000,0.0,7.000000,0.000000,22.000000,45.000000,0.380000,0.139774
50%,7319.00000,1.571832e+15,49.000000,1946.000000,291.000000,22.000000,0.0,31.000000,2.000000,43.000000,49.000000,0.810000,0.149538
75%,10978.50000,1.571854e+15,55.000000,2048.000000,798.000000,30.000000,0.0,42.000000,4.000000,85.000000,55.000000,1.150000,0.389648
max,14638.00000,1.571875e+15,83.000000,2048.000000,1290.000000,62.000000,0.0,90.000000,27.000000,97.000000,83.000000,6.470000,0.629883


In [5]:
data.head()

,Unnamed: 0,id,when,temp_celsius,memory_total_mb,memory_in_use_mb,cpu_user,cpu_background,cpu_system,cpu_interrupt,cpu_idle,cputemp_celsius,load_avg1,memory_pct
0,0,0c8126c71290,1571869087983314,48.0,1945,252,15,0,20,1,63,48.0,0.83,0.129563
1,1,0c8126c7054d,1571869101614331,53.0,1945,272,22,0,41,2,34,53.0,0.82,0.139846
2,2,84b59cc279e0,1571869106377521,56.0,2048,1269,1,0,3,0,96,56.0,0.28,0.619629
3,3,84c1c1d607e0,1571869106399702,54.0,2048,921,6,0,3,0,91,54.0,0.37,0.449707
4,4,84c1c1d607e0,1571869106399702,71.0,2048,798,6,0,2,0,92,71.0,0.37,0.389648


In [23]:

import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username="chengjing1016", api_key="g9cUDdcBEuZ0onVHNopl")

x = data.cpu_user
y = data.temp_celsius

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [98]:

import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username="chengjing1016", api_key="g9cUDdcBEuZ0onVHNopl")

x = data.load_avg1
y = data.cpu_idle

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [21]:

import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username="chengjing1016", api_key="g9cUDdcBEuZ0onVHNopl")

x = data.memory_pct
y = data.cpu_user

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [29]:
x = data.memory_pct
y = data.load_avg1

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [24]:
data['id'].unique()

array(['0c8126c71290', '0c8126c7054d', '84b59cc279e0', '84c1c1d607e0',
       'd8b12288af4f', '0c8126c70665', '28a24b810c97', '3c8c939495ac',
       '80acacf2f530', '0c8126c6e923', '3c8c9394a39e'], dtype=object)

In [80]:
data0 = data[data.id=='d8b12288af4f']

In [81]:

import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username="chengjing1016", api_key="g9cUDdcBEuZ0onVHNopl")

x = data0.cpu_user
y = data0.temp_celsius

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [85]:
x = data0.load_avg1
y = data0.cpu_user

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [83]:
x = data0.memory_pct
y = data0.cpu_user

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [84]:
x = data0.memory_pct
y = data0.load_avg1

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [75]:
data0 = data[data.id=='3c8c9394a39e']

In [76]:
x = data0.cpu_user
y = data0.temp_celsius

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [77]:
x = data0.load_avg1
y = data0.cpu_user

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [78]:
x = data0.memory_pct
y = data0.cpu_user

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [79]:
x = data0.memory_pct
y = data0.load_avg1

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers'
)

data1 = [trace]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='temp vs cpu')


In [88]:
data0.describe()

,Unnamed: 0,when,temp_celsius,memory_total_mb,memory_in_use_mb,cpu_user,cpu_background,cpu_system,cpu_interrupt,cpu_idle,cputemp_celsius,load_avg1,memory_pct
count,1439.000000,1.439000e+03,1439.000000,1439.0,1439.000000,1439.000000,1439.0,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000
mean,7348.195969,1.571832e+15,44.276581,1945.0,266.815844,22.573315,0.0,39.719249,3.285615,34.424600,44.276581,1.909583,0.137180
std,4221.262380,2.493380e+10,1.773299,0.0,8.767044,6.614984,0.0,13.033246,2.215700,19.784366,1.773299,0.844183,0.004507
min,5.000000,1.571789e+15,41.000000,1945.0,252.000000,6.000000,0.0,10.000000,0.000000,0.000000,41.000000,0.380000,0.129563
25%,3673.000000,1.571810e+15,43.000000,1945.0,252.000000,18.000000,0.0,30.000000,2.000000,18.000000,43.000000,1.290000,0.129563
50%,7365.000000,1.571832e+15,44.000000,1945.0,272.000000,23.000000,0.0,40.000000,3.000000,33.000000,44.000000,1.720000,0.139846
75%,10975.500000,1.571854e+15,46.000000,1945.0,272.000000,28.000000,0.0,49.500000,4.000000,49.000000,46.000000,2.420000,0.139846
max,14632.000000,1.571875e+15,47.000000,1945.0,272.000000,38.000000,0.0,72.000000,12.000000,83.000000,47.000000,6.470000,0.139846


In [94]:
for i in data['id'].unique():
    print (i)
    print (data[data.id==i].describe())

0c8126c71290
         Unnamed: 0          when  temp_celsius  memory_total_mb  \
count   1439.000000  1.439000e+03   1439.000000           1439.0   
mean    7355.045865  1.571832e+15     48.624739           1945.0   
std     4201.939436  2.495325e+10      0.484358              0.0   
min        0.000000  1.571789e+15     48.000000           1945.0   
25%     3782.000000  1.571810e+15     48.000000           1945.0   
50%     7278.000000  1.571832e+15     49.000000           1945.0   
75%    11025.000000  1.571854e+15     49.000000           1945.0   
max    14611.000000  1.571875e+15     49.000000           1945.0   

       memory_in_use_mb     cpu_user  cpu_background   cpu_system  \
count       1439.000000  1439.000000          1439.0  1439.000000   
mean         255.798471    23.250174             0.0    36.704656   
std            8.894043     6.373948             0.0    10.706330   
min          252.000000     7.000000             0.0    10.000000   
25%          252.000000    18

         Unnamed: 0          when  temp_celsius  memory_total_mb  \
count   1440.000000  1.440000e+03   1440.000000           1440.0   
mean    7343.566667  1.571832e+15     46.230556           1946.0   
std     4224.196367  2.495034e+10      1.022830              0.0   
min        6.000000  1.571789e+15     44.000000           1946.0   
25%     3662.500000  1.571810e+15     45.750000           1946.0   
50%     7368.000000  1.571832e+15     47.000000           1946.0   
75%    10970.250000  1.571854e+15     47.000000           1946.0   
max    14636.000000  1.571875e+15     48.000000           1946.0   

       memory_in_use_mb     cpu_user  cpu_background   cpu_system  \
count       1440.000000  1440.000000          1440.0  1440.000000   
mean         297.245833    27.143056             0.0    33.904861   
std           16.694803     8.142090             0.0     8.849682   
min          291.000000     7.000000             0.0    10.000000   
25%          291.000000    21.000000      

In [ ]:
dfin = dfin.withColumnRenamed('mac','mac1').withColumnRenamed('when','when1')

df0 = dfeng.join(dfin, (dfeng.mac==dfin.mac1)&(dfeng.when==dfin.when1)&(dfeng.module_index==dfin.re_ix), 'inner') 

In [ ]:
df0 = df0.filter(df0.media_type=='copper').filter(df0.type=='phy').filter(df0.link==True).filter(df0.admin_status==True)
df1 = df0.select('mac','name','time','tx_drops','rx_packets','tx_packets','cpu_user','cpu_idle','temp_celsius','power_pct','memory_pct')
df1.toPandas().to_csv('switch.csv')



In [ ]:
# data smoothing
import numpy

def smooth(x,window_len=10,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y




# from numpy import *
# from pylab import *

# def smooth_demo():

#     t=linspace(-4,4,100)
#     x=sin(t)
#     xn=x+randn(len(t))*0.1
#     y=smooth(x)

#     ws=31

#     subplot(211)
#     plot(ones(ws))

#     windows=['flat', 'hanning', 'hamming', 'bartlett', 'blackman']

#     hold(True)
#     for w in windows[1:]:
#         eval('plot('+w+'(ws) )')

#     axis([0,30,0,1.1])

#     legend(windows)
#     title("The smoothing windows")
#     subplot(212)
#     plot(x)
#     plot(xn)
#     for w in windows:
#         plot(smooth(xn,10,w))
#     l=['original signal', 'signal with noise']
#     l.extend(windows)

#     legend(l)
#     title("Smoothing a noisy signal")
#     show()


# if __name__=='__main__':
#     smooth_demo()

In [ ]:
# Build Baseline
df0.groupby('mac','name','re_ix').agg(avg('temp_celsius').alias('avg_temp'), stddev('temp_celsius').alias('std_temp'), \
                                      avg('memory_pct').alias('avg_memory'), stddev('memory_pct').alias('std_memory'))
upper = udf(lambda x: x[0]+3*x[1]), FloatType())
df0 = df0.withColumn('upper_temp', upper('avg_temp', 'std_temp')) \
            .withColumn('upper_memory', upper('avg_memory', 'std_memory'))

# Smoothing new data: here also use df0 
df0 = df0.select('mac','name','re_ix','temp_celsius','avg_temp','std_temp','memory_pct','avg_memory','std_memory','cpu_user','power_pct')
pdf = df0.toPandas()
temp_l = smooth(pdf['temp_celsius'].tolist())
pdf['temp_s'] = pd.DataFrame(temp_l)
memory_l = smooth(pdf['memory_pct'].tolist())
pdf['memory_s'] = pd.DataFrame(memory_l)
cpuuser_l = smooth(pdf['cpu_user'].tolist())
pdf['cpuuser_s'] = pd.DataFrame(cpuuser_l)
power_l = smooth(pdf['power_pct'].tolist())
pdf['power_s'] = pd.DataFrame(power_l)

# ???
df0 = spark_session.createDataFrame(pdf)

# Yellow AD for new data
yellow = udf(lambda x: 1 if x[0]>x[1] else 0, IntegerType())
df0 = df0.withColumn('yellow_temp', yellow('temp_celsius', 'upper_temp')) \
            .withColumn('yellow_memory', yellow('memory_pct', 'upper_memory'))

# Red Alarm for new data
red = udf(lambda x: 1 if x>=0.9 else 0, IntegerType())
redtemp = udf(lambda x: 1 if x>=70 else 0, IntergerType())
df0 = df0.withColumn('red_temp', redtemp('temp_celsius')) \
            .withColumn('red_memory', red('pct_memory')) \
            .withColumn('red_user', red('cpu_user')) \
            .withColumn('red_power', red('power_pct'))




In [ ]:
date = '2019-10-15'
In [85]: dfin.filter(dfin.link==True).filter(dfin.full_duplex==False).filter(dfin.type=='phy').select('id','name').
    ...: distinct().show()                                                                                         
+------------+---------+                                                        
|          id|     name|
+------------+---------+
|64649b511200| ge-0/0/1|
|64649b509000|ge-0/0/16|
|84b59cc279e0|ge-0/0/11| cuong 
|64649b511200|ge-0/0/15|
|64649b511200|ge-0/0/16|
|64649b509000|ge-0/0/15|
|84b59cc279e0| ge-0/0/7| cuong
|64649b511200| ge-0/0/2|
|84b59cc279e0| ge-0/0/9| cuong
|64649b509000| ge-0/0/1|
|64649b509000|ge-0/0/14|
|64649b509000| ge-0/0/2|
|84b59cc279e0| ge-0/0/3| cuong
+------------+---------+

In [ ]:
# Fix the followings w new schema

In [1]:
#flatten chassis: flatten all 4 arrays
dfch = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.chassis.routing_engines').alias('routing_engines1'),
                 col('device.chassis.fans').alias('fans1'),
                 col('device.chassis.temperatures').alias('temperatures1'),
                 col('device.chassis.psus').alias('psus1')
                )

dfch = dfch.select(col('*'),explode_outer('routing_engines1').alias('routing_engines')) \
            .select(col('*'),explode_outer('temperatures1').alias('temperatures')) \
            .select(col('*'),explode_outer('fans1').alias('fans')) \
            .select(col('*'),explode_outer('psus1').alias('psus')) \
            .drop('routing_engines1','temperatures1','fans1','psus1')

dfch = dfch.select(col('*'),
                  col('routing_engines.slot').alias('eng_slot'),
                  col('routing_engines.is_master').alias('eng_ismaster'),
                  col('routing_engines.status').alias('eng_status'),
                  col('routing_engines.temp_celsius').alias('eng_tempc'),
                  col('routing_engines.memory.total_mb').alias('eng_memoryttlmb'),
                  col('routing_engines.memory.in_use_mb').alias('eng_memoryinusemb'),
                  col('routing_engines.cpu.user').alias('eng_cpuuser'),
                  col('routing_engines.cpu.background').alias('eng_cpubackground'),
                  col('routing_engines.cpu.system').alias('eng_cpusystem'),
                  col('routing_engines.cpu.interrupt').alias('eng_cpuinterrupt'),
                  col('routing_engines.cpu.idle').alias('eng_cpuidle'),
                  col('routing_engines.cputemp_celsius').alias('eng_cputempc'),
                  col('routing_engines.model').alias('eng_model'),
                  col('routing_engines.last_reboot_reason').alias('eng_lastrebootreason'),
                  col('routing_engines.load_avg').alias('eng_loadavg'),
                  col('fans.name').alias('fans_name'),
                  col('fans.status').alias('fans_status'),
                   col('temperatures.name').alias('temp_name'),
                   col('temperatures.status').alias('temp_status'),
                   col('temperatures.temp_celsius').alias('temp_tempc'),
                    col('psus.name').alias('psus_name'),
                   col('psus.status').alias('psus_status')) \
                    .drop('routing_engines','fans','temperatures','psus')

dfch.persist().first()
dfch.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/chassis'+date)
dfch.unpersist()

NameError: name 'df' is not defined

In [ ]:
#flatten config_history
dfconf = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.config_history').alias('config_history1')
                )

dfconf = dfconf.select(col('*'),explode_outer('config_history1').alias('config_history')) \
            .drop('config_history1')

dfconf = dfconf.select(col('*'),
                  col('config_history.sequence_num').alias('confhist_seqnum'),
                  col('config_history.user').alias('confhist_user'),
                   col('config_history.client').alias('confhist_client'),
                   col('config_history.diff').alias('confhist_diff'),
                   col('config_history.timestamp').alias('confhist_timestamp')) \
                    .drop('config_history')

dfconf.persist().first()
dfconf.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/config_history'+date)
dfconf.unpersist()

In [ ]:
#flatten alarms
dfalarm = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.alarms').alias('alarms1')
                )

dfalarm = dfalarm.select(col('*'),explode_outer('alarms1').alias('alarms')) \
            .drop('alarms1')

dfalarm = dfalarm.select(col('*'),
                  col('alarms.timestamp').alias('alarm_ts'),
                  col('alarms.type').alias('alarm_type'),
                   col('alarms.alarm_class').alias('alarm_class'),
                   col('alarms.description').alias('alarm_desc'),
                   col('alarms.short_description').alias('alarm_shortdesc')) \
                    .drop('alarms')

dfalarm.persist().first()
dfalarm.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/alarms'+date)
dfalarm.unpersist()

In [ ]:
#flatten ipv4_route
dfipv = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.ipv4_route').alias('ipv1')
                )

dfipv = dfipv.select(col('*'),explode_outer('ipv1').alias('ipv')) \
            .drop('ipv1')

dfipv = dfipv.select(col('*'),
                  col('ipv.dst.mask').alias('ipv_dstmask'),
                  col('ipv.dst.ip').alias('ipv_dstip'),
                   col('ipv.gw').alias('ipv_gw')) \
                    .drop('ipv')

dfipv.persist().first()
dfipv.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/ipv'+date)
dfipv.unpersist()

In [ ]:
#flatten lldpneighbors
dflldp = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.lldpneighbors').alias('lldp1')
                )

dflldp = dflldp.select(col('*'),explode_outer('lldp1').alias('lldp')) \
            .drop('lldp1')

dflldp = dflldp.select(col('*'),
                  col('lldp.local_port_id').alias('lldp_locportid'),
                  col('lldp.local_parent_iface_name').alias('lldp_locifacename'),
                   col('lldp.remote_chassis_id').alias('lldp_rmtchassisid'),
                   col('lldp.remote_chassis_idsubtype').alias('lldp_rmtchassisidsub'),
                      col('lldp.remote_port_desc').alias('lldp_rmtportdesc'),
                      col('lldp.remote_system_name').alias('lldp_rmtsysname'),
                      col('lldp.remote_mgmt_addr').alias('lldp_rmtmgmtaddr'),
                      col('lldp.remote_system_desc').alias('lldp_rmtsysdesc')) \
                        .drop('lldp')

dflldp.persist().first()
dflldp.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/lldp'+date)
dflldp.unpersist()

In [ ]:
#flatten svistats
dfsvi = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').a\lias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.svistats').alias('svi1')
                )

dfsvi = dfsvi.select(col('*'),explode_outer('svi1').alias('svi')) \
            .drop('svi1')

dfsvi = dfsvi.select(col('*'),
                  col('svi.dev').alias('svi_dev'),
                  col('svi.vlan').alias('svi_vlan'),
                   col('svi.ips').alias('svi_ips')) \
                    .drop('svi')

dfsvi.persist().first()
dfsvi.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/svi'+date)
dfsvi.unpersist()

# Analysis

In [ ]:
+-----------------+
|64-64-9B-50-90-00|
|0C-81-26-C6-FF-6C|
|84-B5-9C-C2-79-E0|
|0C-81-26-C6-EC-25|
|3C-8C-93-94-95-AC|
|D8-B1-22-88-AF-4F|
|64-64-9B-51-12-00|
|0C-81-26-C7-05-4D|
|C8-E7-F0-FC-72-E7|
|28-A2-4B-81-0C-97|
|0C-81-26-C7-12-90|
|E8-B6-C2-60-10-E0|
+-----------------+

In [ ]:
In [16]: dfch.select('id','site_id','model','version','eng_model').distinct().sort('site_id').take(20)                                   
Out[16]:                                                                        
[Row(id='0C-81-26-C6-E9-23', site_id='', model='EX2300-C-12P', version='19.2R1.8', eng_model='RE-EX2300C-12P'),
 Row(id='0C-81-26-C6-EC-25',  site_id='', model='EX2300-C-12P', version='18.1R3.3', eng_model='RE-EX2300C-12P'),
 Row(id='3C-8C-93-94-95-AC',  site_id='12c07c11-51a3-4958-9aee-8bf0f7ff193c', model='EX2300-C-12P', version='18.1R3.3', eng_model='RE-EX2300C-12P'), raymond
 Row(id='80-AC-AC-F2-F5-30', site_id='1916d52a-4a90-11e5-8b45-1258369c38a9', model='EX2300-C-12P', version='19.2R1.8', eng_model='RE-EX2300C-12P'), linker
 Row(id='64-64-9B-50-90-00',  site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', model='EX4600-40F', version='18.1R3-S5.3', eng_model='EX4600-40F'),   walmart
 Row(id='0C-81-26-C7-05-4D',  site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', model='EX2300-C-12P', version='18.1R3.3', eng_model='RE-EX2300C-12P'), abhi desk
 Row(id='0C-81-26-C7-12-90',  site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', model='EX2300-C-12P', version='18.1R3.3', eng_model='RE-EX2300C-12P'), abhi lab
 Row(id='E8-B6-C2-60-10-E0',  site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', model='EX2300-48P', version='18.2R3-S1.7', eng_model='RE-EX2300-48P'), walmart
 Row(id='C8-E7-F0-FC-72-E7',  site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', model='EX2300-48P', version='18.2R3-S1.7', eng_model='RE-EX2300-48P'), walmart
 Row(id='64-64-9B-51-12-00',  site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', model='EX4600-40F', version='18.1R3-S5.3', eng_model='EX4600-40F'),  walmart
 Row(id='28-A2-4B-81-0C-97',  site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', model='EX2300-24P', version='18.1R3.3', eng_model='RE-EX2300-24P'), yh
 Row(id='D8-B1-22-88-AF-4F',  site_id='67970e46-4e12-11e6-9188-0242ac110007', model='EX2300-C-12P', version='18.2R2.6', eng_model='RE-EX2300C-12P'), mist office
 Row(id='0C-81-26-C6-FF-6C',  site_id='c1698122-c14c-11e5-8e81-1258369c38a9', model='EX2300-C-12P', version='18.1R3.3', eng_model='RE-EX2300C-12P'), cuong
 Row(id='5C-45-27-A9-65-80', site_id='c1698122-c14c-11e5-8e81-1258369c38a9', model='EX3400-48P', version='19.2R1.8', eng_model='RE-EX3400-48P'), cuong
 Row(id='84-B5-9C-C2-79-E0',  site_id='c1698122-c14c-11e5-8e81-1258369c38a9', model='EX4300-48P', version='19.1R1.6', eng_model='EX4300-48P'), cuong
 Row(id='0C-81-26-C7-06-65', site_id='fa438196-e1ae-11e5-bf9e-1258369c38a9', model='EX2300-C-12P', version='19.2R1.8', eng_model='RE-EX2300C-12P'), jerry
 Row(id='84-C1-C1-D6-07-E0', site_id='fa438196-e1ae-11e5-bf9e-1258369c38a9', model='EX4300-48P', version='14.1X53-D35.3', eng_model='EX4300-48P')] jerry


In [17]: dfch.groupby('site_id').agg(countDistinct('id')).show()                                                                         
+--------------------+------------------+                                       
|             site_id|count(DISTINCT id)|
+--------------------+------------------+
|5fda4ac7-62b0-458...|                 1|
|67970e46-4e12-11e...|                 1|
|fa438196-e1ae-11e...|                 2|
|12c07c11-51a3-495...|                 1|
|1916d52a-4a90-11e...|                 1|
|57b2f891-09c1-4dc...|                 6|
|                    |                 2|
|c1698122-c14c-11e...|                 3|
+--------------------+------------------+


In [18]: dfch.groupby('model').agg(countDistinct('id')).show()                                                                           
+------------+------------------+                                               
|       model|count(DISTINCT id)|
+------------+------------------+
|  EX4600-40F|                 2|
|  EX4300-48P|                 2|
|EX2300-C-12P|                 9|
|  EX2300-24P|                 1|
|  EX2300-48P|                 2|
|  EX3400-48P|                 1|
+------------+------------------+


In [19]: dfch.groupby('version').agg(countDistinct('id')).show()                                                                         
+-------------+------------------+                                              
|      version|count(DISTINCT id)|
+-------------+------------------+
|  18.2R3-S1.7|                 2|
|     19.2R1.8|                 4|
|14.1X53-D35.3|                 1|
|     18.1R3.3|                 6|
|     18.2R2.6|                 1|
|  18.1R3-S5.3|                 2|
|     19.1R1.6|                 1|
+-------------+------------------+


In [20]: dfch.groupby('eng_model').agg(countDistinct('id')).show()                                                                       
+--------------+------------------+                                             
|     eng_model|count(DISTINCT id)|
+--------------+------------------+
| RE-EX3400-48P|                 1|
|    EX4600-40F|                 2|
|    EX4300-48P|                 2|
|RE-EX2300C-12P|                 9|
| RE-EX2300-48P|                 2|
| RE-EX2300-24P|                 1|
+--------------+------------------+



In [ ]:
# Rule 1: bad cable
In [12]: dfin.groupby('admin_status','link','poe_status').count().show()                                          
+------------+-----+----------+------+                                          
|admin_status| link|poe_status| count| + poe_power > 0 
+------------+-----+----------+------+
|       false|false|     false|  1439| cable down? 
|        true|false|      true|    12| ** Bad Cable (AD) / charging
|        true| true|      true| 47452| ok
|        true|false|     false|396947| laptop
|        true| true|     false|495610| ?
+------------+-----+----------+------+

dfin.filter(col('admin_status')==True).filter(col('link')==False).filter(col('poe_status')==True).filter(col('poe_power')>0).count() 
# 12 but some device just for charging
#Q: bad cable? but there are pkts...
[Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569230760790265, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8174843, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/6', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa0'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569269280970968, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8213363, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/6', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa0'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='0C-81-26-C7-06-65', mac='0C-81-26-C7-06-65', org_id='41a0d1a4-e157-11e5-9ffd-1258369c38a9', site_id='fa438196-e1ae-11e5-bf9e-1258369c38a9', ts=1569212213864412, remote_addr=bytearray(b'd`F\x00\x10U'), hostname='jerry-ex2300', model='EX2300-C-12P', version='19.2R1.8', serial='HV3619230216', uptime=371344, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/0', link=False, full_duplex=True, mbps=1000, mtu=1514, address=bytearray(b'\x0c\x81&\xc7\x06h'), admin_status=True, last_flapped=1569212215, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=1.2000000476837158, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=22, tx_pkts=941, rx_bytes=2808, tx_bytes=107591),
 Row(id='0C-81-26-C7-06-65', mac='0C-81-26-C7-06-65', org_id='41a0d1a4-e157-11e5-9ffd-1258369c38a9', site_id='fa438196-e1ae-11e5-bf9e-1258369c38a9', ts=1569212273864624, remote_addr=bytearray(b'd`F\x00\x10U'), hostname='jerry-ex2300', model='EX2300-C-12P', version='19.2R1.8', serial='HV3619230216', uptime=371404, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/0', link=False, full_duplex=True, mbps=1000, mtu=1514, address=bytearray(b'\x0c\x81&\xc7\x06h'), admin_status=True, last_flapped=1569212243, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=1.399999976158142, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=4, tx_pkts=8, rx_bytes=402, tx_bytes=1329),
 Row(id='0C-81-26-C7-06-65', mac='0C-81-26-C7-06-65', org_id='41a0d1a4-e157-11e5-9ffd-1258369c38a9', site_id='fa438196-e1ae-11e5-bf9e-1258369c38a9', ts=1569212333864841, remote_addr=bytearray(b'd`F\x00\x10U'), hostname='jerry-ex2300', model='EX2300-C-12P', version='19.2R1.8', serial='HV3619230216', uptime=371464, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/0', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'\x0c\x81&\xc7\x06h'), admin_status=True, last_flapped=1569212280, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=1.600000023841858, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=4, tx_pkts=2, rx_bytes=625, tx_bytes=399),
 Row(id='0C-81-26-C7-06-65', mac='0C-81-26-C7-06-65', org_id='41a0d1a4-e157-11e5-9ffd-1258369c38a9', site_id='fa438196-e1ae-11e5-bf9e-1258369c38a9', ts=1569232133939480, remote_addr=bytearray(b'd`F\x00\x10U'), hostname='jerry-ex2300', model='EX2300-C-12P', version='19.2R1.8', serial='HV3619230216', uptime=391265, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/0', link=False, full_duplex=True, mbps=1000, mtu=1514, address=bytearray(b'\x0c\x81&\xc7\x06h'), admin_status=True, last_flapped=1569232082, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=1.600000023841858, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=4, tx_pkts=3, rx_bytes=402, tx_bytes=467),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569230760790265, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8174843, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/9', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa3'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569269280970968, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8213363, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/9', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa3'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569230760790265, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8174843, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/8', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa2'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569269280970968, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8213363, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/8', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa2'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569230760790265, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8174843, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/7', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa1'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0),
 Row(id='28-A2-4B-81-0C-97', mac='28-A2-4B-81-0C-97', org_id='df59193d-ecdc-4a07-b570-b2de6c62c464', site_id='5fda4ac7-62b0-4588-95b3-f8234e890b13', ts=1569269280970968, remote_addr=bytearray(b'd`D\x00\xea>'), hostname='yhlee-ex2300-24p', model='EX2300-24P', version='18.1R3.3', serial='JY0216490819', uptime=8213363, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, name='ge-0/0/7', link=False, full_duplex=False, mbps=1000, mtu=1514, address=bytearray(b'(\xa2K\x81\x0c\xa1'), admin_status=True, last_flapped=247, errors=[], poe_enabled=True, poe_status=True, poe_pwrlimit=15.399999618530273, poe_power=0.0, poe_priority=False, poe_class='0', poe_mode='', rx_bps=0, tx_bps=0, rx_pkts=0, tx_pkts=0, rx_bytes=0, tx_bytes=0)]


In [ ]:
# Rule 2: fans faliure

In [22]: dfch.groupby('id').agg(countDistinct('fans_name')).show()                                                                       
+-----------------+-------------------------+                                   
|               id|count(DISTINCT fans_name)|
+-----------------+-------------------------+
|64-64-9B-50-90-00|                        1|
|0C-81-26-C6-FF-6C|                        0|
|5C-45-27-A9-65-80|                        2|
|0C-81-26-C6-E9-23|                        0|
|0C-81-26-C7-06-65|                        0|
|84-B5-9C-C2-79-E0|                        4|
|0C-81-26-C6-EC-25|                        0|
|3C-8C-93-94-95-AC|                        0|
|D8-B1-22-88-AF-4F|                        0|
|64-64-9B-51-12-00|                        1|
|0C-81-26-C7-05-4D|                        0|
|84-C1-C1-D6-07-E0|                        8|
|C8-E7-F0-FC-72-E7|                        2|
|28-A2-4B-81-0C-97|                        2|
|80-AC-AC-F2-F5-30|                        0|
|0C-81-26-C7-12-90|                        0|
|E8-B6-C2-60-10-E0|                        2|
+-----------------+-------------------------+

In [24]: dfch.groupby('model').agg(countDistinct('fans_name')).show()                                                                    
+------------+-------------------------+                                        
|       model|count(DISTINCT fans_name)|
+------------+-------------------------+
|  EX4600-40F|                        1|
|  EX4300-48P|                        8|
|EX2300-C-12P|                        0|
|  EX2300-24P|                        2|
|  EX2300-48P|                        2|
|  EX3400-48P|                        2|
+------------+-------------------------+

In [26]: dfch0.filter(dfch0.fans_status==False).count()                                                                                  
Out[26]: 75744                                                                  

In [27]: dfch0.filter(dfch0.fans_status==True).count()                                                                                   
Out[27]: 307369   
    
dfch0 = dfch.groupby('id','ts').agg(max_('fans_status'))     
dfch1 = dfch0.groupby('id').agg(max_('max(fans_status)'))     
dfch1.filter(col('max(max(fans_status))')==False).count()      
# 0 means all switches have at least one fan (model without fans is exception)

In [ ]:
# last_rebootreason:
[Row(eng_lastrebootreason='0x40000:Mac Reset'), 12k
 Row(eng_lastrebootreason='0x1:power cycle/failure'), 103k
 Row(eng_lastrebootreason='Router rebooted after a normal shutdown.')] 975k"

In [ ]:
# Check alarms about fans

In [29]: dfalarm.select('id','alarm_ts','alarm_type','alarm_class','alarm_desc').filter(col('alarm_type').isNotNull()).distinct().take(20
    ...: )                                                                                                                               
Out[29]:                                                                        
[Row(id='84-C1-C1-D6-07-E0', alarm_ts=Row(seconds=1569256743, nanos=0), alarm_type='Chassis', alarm_class='Major', alarm_desc='FPC 1 PSU 0 Output Failure'),
 Row(id='5C-45-27-A9-65-80', alarm_ts=Row(seconds=1299633273, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='RE 0 /var partition usage is high'),
 x Row(id='84-C1-C1-D6-07-E0', alarm_ts=Row(seconds=1569258218, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='FPC 0 Fan/Blower Absent'),
 Row(id='0C-81-26-C7-06-65', alarm_ts=Row(seconds=1568841657, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='RE 0 /var partition usage is high'),
 Row(id='64-64-9B-50-90-00', alarm_ts=Row(seconds=1568046357, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='FPC 0 PEM 0 is not powered'),
 x Row(id='84-C1-C1-D6-07-E0', alarm_ts=Row(seconds=1569262715, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='FPC 1 Fan/Blower Absent'),
 x Row(id='28-A2-4B-81-0C-97', alarm_ts=Row(seconds=359, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='FPC 0 Fan 1 Fan/Blower Absent'),
 Row(id='84-C1-C1-D6-07-E0', alarm_ts=Row(seconds=1569256738, nanos=0), alarm_type='Chassis', alarm_class='Major', alarm_desc='FPC 1 PSU 0 Not OK'),
 x Row(id='84-C1-C1-D6-07-E0', alarm_ts=Row(seconds=1569268235, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='FPC 0 Fan/Blower Absent'),
 Row(id='64-64-9B-51-12-00', alarm_ts=Row(seconds=1568305703, nanos=0), alarm_type='Chassis', alarm_class='Minor', alarm_desc='FPC 0 PEM 0 is not powered')]

In [32]: dfch.filter(col('id')=='84-C1-C1-D6-07-E0').groupby('fans_name','fans_status').count().show()                                   
+-------------------+-----------+-----+                                         
|          fans_name|fans_status|count|
+-------------------+-----------+-----+
|FPC 0 Fan 0 Airflow|       true|42240| 
|FPC 1 Fan 1 Airflow|       true|42240|
|        FPC 0 Fan 0|       true|42240|
|FPC 1 Fan 0 Airflow|       true| 9408|
|FPC 0 Fan 1 Airflow|       true| 2208|
|        FPC 0 Fan 1|      false|40032| x alarm reported missing twice
|        FPC 1 Fan 0|      false|32832| x alarm reported missing
|        FPC 1 Fan 0|       true| 9408| x fix later
|        FPC 0 Fan 1|       true| 2208| x fix in the middle
|        FPC 1 Fan 1|       true|42240|
+-------------------+-----------+-----+

In [35]: dfch.filter(col('id')=='28-A2-4B-81-0C-97').groupby('fans_name','fans_status').count().show()                                   
+----------------+-----------+-----+                                            
|       fans_name|fans_status|count|
+----------------+-----------+-----+
|FPC 0 Fan Tray 0|       true| 2880|
|FPC 0 Fan Tray 1|      false| 2880| 
+----------------+-----------+-----+


In [ ]:
# Rule 3: psus unplug

In [40]: dfch.groupby('psus_name','psus_status').count().show()                                                                          
+--------------------+-----------+------+                                       
|           psus_name|psus_status| count|
+--------------------+-----------+------+
|FPC 1 Power Supply 1|       true| 66264|
|FPC 1 Power Supply 0|      false| 66264|
|FPC 0 Power Supply 0|      false| 20153|
|FPC 0 Power Supply 1|      false|106580|
|FPC 0 Power Supply 0|       true|143732|
+--------------------+-----------+------+

In [8]: dfch.groupby('id').agg(countDistinct('psus_name'),countDistinct('psus_status')).show()                     
+-----------------+-------------------------+---------------------------+       
|               id|count(DISTINCT psus_name)|count(DISTINCT psus_status)|
+-----------------+-------------------------+---------------------------+
|64-64-9B-50-90-00|                        1|                          1|
|0C-81-26-C6-FF-6C|                        1|                          1|
|5C-45-27-A9-65-80|                        2|                          2|
|0C-81-26-C6-E9-23|                        1|                          1|
|0C-81-26-C7-06-65|                        1|                          1|
|84-B5-9C-C2-79-E0|                        2|                          2|
|0C-81-26-C6-EC-25|                        1|                          1|
|3C-8C-93-94-95-AC|                        1|                          1|
|D8-B1-22-88-AF-4F|                        1|                          1|
|64-64-9B-51-12-00|                        1|                          1|
|0C-81-26-C7-05-4D|                        1|                          1|
|84-C1-C1-D6-07-E0|                        4|                          2|
|C8-E7-F0-FC-72-E7|                        1|                          1|
|28-A2-4B-81-0C-97|                        1|                          1|
|0C-81-26-C7-12-90|                        1|                          1|
|E8-B6-C2-60-10-E0|                        1|                          1|
+-----------------+-------------------------+---------------------------+

dfch0 = dfch.groupby('id','ts').agg(max_('psus_status'))   
dfch1 = dfch0.groupby('id').agg(max_('max(psus_status)'))     
dfch1.filter(col('max(max(psus_status))')==False).count()    
# 2    
+-----------------+---------------------+                                       
|               id|max(max(psus_status))|
+-----------------+---------------------+
|64-64-9B-50-90-00|                false|
|0C-81-26-C6-FF-6C|                 true|
|5C-45-27-A9-65-80|                 true|
|0C-81-26-C6-E9-23|                 true|
|0C-81-26-C7-06-65|                 true|
|84-B5-9C-C2-79-E0|                 true|
|0C-81-26-C6-EC-25|                 true|
|3C-8C-93-94-95-AC|                 true|
|D8-B1-22-88-AF-4F|                 true|
|64-64-9B-51-12-00|                false|
|0C-81-26-C7-05-4D|                 true|
|84-C1-C1-D6-07-E0|                 true|
|C8-E7-F0-FC-72-E7|                 true|
|28-A2-4B-81-0C-97|                 true|
|0C-81-26-C7-12-90|                 true|
|E8-B6-C2-60-10-E0|                 true|
+-----------------+---------------------+

In [21]: dfch.filter(col('id')=='64-64-9B-50-90-00').select('psus_name','psus_status').distinct().show()                                 
+--------------------+-----------+                                              
|           psus_name|psus_status|
+--------------------+-----------+
|FPC 0 Power Supply 0|      false|
+--------------------+-----------+ ?? psus is false while there is cpu load_avg and pkts
In [20]: dfch.filter(col('id')=='64-64-9B-50-90-00').first()                                                                             
Out[20]: Row(id='64-64-9B-50-90-00', mac='64-64-9B-50-90-00', org_id='b4e16c72-d50e-4c03-a952-a3217e231e2c', site_id='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39', ts=1569289954801400, remote_addr=bytearray(b'dan\x00\xcb]'), hostname='EX4600-s-dist-1.s00004.us', model='EX4600-40F', version='18.1R3-S5.3', serial='TC3714280029', uptime=1243832, pctrl_ix=0, pctrl_maxpwr=0.0, pctrl_consump=0.0, pctrl_gband=0.0, eng_slot=0, eng_ismaster=True, eng_status=True, eng_tempc=37.0, eng_memoryttlmb=1920, eng_memoryinusemb=1056, eng_cpuuser=5, eng_cpubackground=0, eng_cpusystem=3, eng_cpuinterrupt=1, eng_cpuidle=92, eng_cputempc=37.0, eng_model='EX4600-40F', eng_lastrebootreason='0x1:power cycle/failure', eng_loadavg=[0.1899999976158142, 0.07000000029802322, 0.05000000074505806], fans_name='FPC 0 Fan Tray 0', fans_status=True, temp_name='FPC 0 Sensor TopMiddle C', temp_status=True, temp_tempc=37.0, psus_name='FPC 0 Power Supply 0', psus_status=False)  
    

In [ ]:
# Features
dfin.groupby('id','name').agg(min_('rx_pkts'),max_('rx_pkts'),avg('rx_pkts'),stddev('rx_pkts'),min_('tx_pkts'),max_('tx_pkts'),avg('tx_pkts'),stddev('tx_pkts'),min_('poe_power'),max_('poe_power'),avg('poe_power'),stddev('poe_power'),min_('pctrl_consump'),max_('pctrl_consump'),avg('pctrl_consump'),stddev('pctrl_consump')) 

'f1.csv'

dfin.groupby('id').agg(min_('rx_pkts'),max_('rx_pkts'),avg('rx_pkts'),stddev('rx_pkts'),min_('tx_pkts'),max_('tx_pkts'),avg('tx_pkts'),stddev('tx_pkts'),min_('poe_power'),max_('poe_power'),avg('poe_power'),stddev('poe_power'),min_('pctrl_consump'),max_('pctrl_consump'),avg('pctrl_consump'),stddev('pctrl_consump')) 
+-----------------+------------+------------+------------------+--------------------+------------+------------+------------------+--------------------+--------------+--------------+--------------------+----------------------+------------------+------------------+------------------+--------------------------+
|               id|min(rx_pkts)|max(rx_pkts)|      avg(rx_pkts)|stddev_samp(rx_pkts)|min(tx_pkts)|max(tx_pkts)|      avg(tx_pkts)|stddev_samp(tx_pkts)|min(poe_power)|max(poe_power)|      avg(poe_power)|stddev_samp(poe_power)|min(pctrl_consump)|max(pctrl_consump)|avg(pctrl_consump)|stddev_samp(pctrl_consump)|
+-----------------+------------+------------+------------------+--------------------+------------+------------+------------------+--------------------+--------------+--------------+--------------------+----------------------+------------------+------------------+------------------+--------------------------+
|64-64-9B-50-90-00|           0|       59511|112.42547904727793|    584.693893056717|           0|       59511|121.46601898280802|   562.7381034302235|           0.0|           0.0|                 0.0|                   0.0|               0.0|               0.0|               0.0|                       0.0|
|0C-81-26-C6-FF-6C|           0|       13597|179.17698573478518|   836.8026245062941|           0|       12946|306.87142314510004|   945.8207284645295|           0.0|           5.6| 0.30910779886067824|    1.2191087003462258|               0.0|               0.0|               0.0|                       0.0|
|5C-45-27-A9-65-80|           0|       12573| 134.9269113704622|   721.2248613934509|           0|        4790|  73.2177541271089|   384.4155380902289|           0.0|           0.0|                 0.0|                   0.0|               0.0|               0.0|               0.0|                       0.0|
|0C-81-26-C6-E9-23|           0|       17954| 508.2929292929293|  2354.3823074526917|           0|       14494| 426.2060606060606|  1864.8109166947527|           0.0|           0.0|                 0.0|                   0.0|               0.0|               0.0|               0.0|                       0.0|
|0C-81-26-C7-06-65|           0|       38125|395.70770947838355|  1274.5304036885318|           0|       38206| 582.8933602880788|  1409.3745699682408|           0.0|          11.6| 0.34437214246629544|    1.6504370572585003|               0.0|               0.0|               0.0|                       0.0|
|84-B5-9C-C2-79-E0|           0|       12898| 92.62784300089994|   591.7217582695648|           0|        5247|  61.1950421336824|   390.1404031029098|           0.0|           0.0|                 0.0|                   0.0|               0.0|               0.0|               0.0|                       0.0|
|0C-81-26-C6-EC-25|           0|       15432| 176.0390544273634|   800.4497938095548|           0|       12651|457.26049640454653|   1139.561051654944|           0.0|          12.8|  1.3560363690958348|     3.336377832004551|               0.0|               0.0|               0.0|                       0.0|
|3C-8C-93-94-95-AC|           0|      193287|194.37888290571217|   1341.445550710508|           0|      191499|330.31073804244534|   1403.744795627015|           0.0|          15.8|  0.8850723316042391|    2.9928598288347423|               0.0|               0.0|               0.0|                       0.0|
|D8-B1-22-88-AF-4F|           0|       27544|298.96581142434803|  1451.5223476496722|           0|       19610| 291.6321191004118|  1416.5176517209702|           0.0|           0.0|                 0.0|                   0.0|               0.0|               0.0|               0.0|                       0.0|
|64-64-9B-51-12-00|           0|       59587|102.05751633986928|   547.9123846164065|           0|       59587| 101.5223908918406|   540.3138521636923|           0.0|           0.0|                 0.0|                   0.0|               0.0|               0.0|               0.0|                       0.0|
|0C-81-26-C7-05-4D|           0|       66715| 167.1576915653349|    917.723918228962|           0|       65832|  251.875699671878|   955.6798659869287|           0.0|          12.0|  0.2943618852194487|    1.7408322770456217|               0.0|               0.0|               0.0|                       0.0|
|84-C1-C1-D6-07-E0|           0|       34627|213.86863658262405|  1226.9690491464005|           0|       14886|229.20868499782904|  1030.9208681614787|           0.0|           8.4| 0.07047564467375031|     0.759179989752372|               0.0|               0.0|               0.0|                       0.0|
|C8-E7-F0-FC-72-E7|           0|      269492| 716.4696578305349|  3143.0267818320576|           0|     1004590|1329.9893289870226|   6913.275660425755|           0.0|           8.2|  0.4281095599381298|    1.7524372280169318|               0.0|               0.0|               0.0|                       0.0|
|28-A2-4B-81-0C-97|           0|        3889| 139.2034640068043|   528.4538678477796|           0|       17563| 375.1354210160056|  1609.4814346685064|           0.0|          41.8|0.002403154698452...|    0.2690285909332486|               0.0|               0.0|               0.0|                       0.0|
|0C-81-26-C7-12-90|x          0|      785234|11352.051293422026|   61476.04110869727|           0|      783607|11402.472917326575|  61349.209632233506|           0.0|          14.2|  1.5851208903220346|     3.773657677224288|               0.0|               0.0|               0.0|                       0.0|
|E8-B6-C2-60-10-E0|           0|       96806| 747.3022673613938|  1939.2430208537862|           0|     1001427|1414.5009667833592|   6358.079964948666|           0.0|           8.2|  1.2718330137622693|    2.8209860166120655|               0.0|               0.0|               0.0|                       0.0|
+-----------------+------------+------------+------------------+--------------------+------------+------------+------------------+--------------------+--------------+--------------+--------------------+----------------------+------------------+------------------+------------------+--------------------------+


dfin.agg(min_('rx_pkts'),max_('rx_pkts'),avg('rx_pkts'),stddev('rx_pkts'),min_('tx_pkts'),max_('tx_pkts'),avg('tx_pkts'),stddev('tx_pkts'),min_('poe_power'),max_('poe_power'),avg('poe_power'),stddev('poe_power'),min_('pctrl_consump'),max_('pctrl_consump'),avg('pctrl_consump'),stddev('pctrl_consump')) 
+------------+------------+------------------+--------------------+------------+------------+-----------------+--------------------+--------------+--------------+------------------+----------------------+------------------+------------------+------------------+--------------------------+
|min(rx_pkts)|max(rx_pkts)|      avg(rx_pkts)|stddev_samp(rx_pkts)|min(tx_pkts)|max(tx_pkts)|     avg(tx_pkts)|stddev_samp(tx_pkts)|min(poe_power)|max(poe_power)|    avg(poe_power)|stddev_samp(poe_power)|min(pctrl_consump)|max(pctrl_consump)|avg(pctrl_consump)|stddev_samp(pctrl_consump)|
+------------+------------+------------------+--------------------+------------+------------+-----------------+--------------------+--------------+--------------+------------------+----------------------+------------------+------------------+------------------+--------------------------+
|           0|      785234|1180.0497526947947|   17851.72168585936|           0|      783607|1237.203474796829|  17820.127716383613|           0.0|          41.8|0.6943952736094295|    2.3810774935867505|               0.0|               0.0|               0.0|                       0.0|
+------------+------------+------------------+--------------------+------------+------------+-----------------+--------------------+--------------+--------------+------------------+----------------------+------------------+------------------+------------------+--------------------------+




In [ ]:
dfch1 = dfch.withColumn('eng_loadavg1', dfch.eng_loadavg[0])
dfch1.groupby('id').agg(min_('eng_loadavg1'),max_('eng_loadavg1'),avg('eng_loadavg1'),stddev('eng_loadavg1'), \
                    min_('eng_tempc'),max_('eng_tempc'),avg('eng_tempc'),stddev('eng_tempc'), \
                    min_('eng_cputempc'),max_('eng_cputempc'),avg('eng_cputempc'),stddev('eng_cputempc'), \
                    min_('temp_tempc'),max_('temp_tempc'),avg('temp_tempc'),stddev('temp_tempc'), \
                    min_('eng_memoryinusemb'),max_('eng_memoryinusemb'),avg('eng_memoryinusemb'),stddev('eng_memoryinusemb'), \
                    min_('eng_cpuuser'),max_('eng_cpuuser'),avg('eng_cpuuser'),stddev('eng_cpuuser'), \
                    min_('eng_cpubackground'),max_('eng_cpubackground'),avg('eng_cpubackground'),stddev('eng_cpubackground'), \
                    min_('eng_cpusystem'),max_('eng_cpusystem'),avg('eng_cpusystem'),stddev('eng_cpusystem'), \
                    min_('eng_cpuinterrupt'),max_('eng_cpuinterrupt'),avg('eng_cpuinterrupt'),stddev('eng_cpuinterrupt'), \
                    min_('eng_cpuidle'),max_('eng_cpuidle'),avg('eng_cpuidle'),stddev('eng_cpuidle'))

+-----------------+-----------------+-----------------+-------------------+-------------------------+--------------+--------------+------------------+----------------------+-----------------+-----------------+------------------+-------------------------+---------------+---------------+------------------+-----------------------+----------------------+----------------------+----------------------+------------------------------+----------------+----------------+------------------+------------------------+----------------------+----------------------+----------------------+------------------------------+------------------+------------------+------------------+--------------------------+---------------------+---------------------+---------------------+-----------------------------+----------------+----------------+-------------------+------------------------+
|               id|min(eng_loadavg1)|max(eng_loadavg1)|  avg(eng_loadavg1)|stddev_samp(eng_loadavg1)|min(eng_tempc)|max(eng_tempc)|    avg(eng_tempc)|stddev_samp(eng_tempc)|min(eng_cputempc)|max(eng_cputempc)| avg(eng_cputempc)|stddev_samp(eng_cputempc)|min(temp_tempc)|max(temp_tempc)|   avg(temp_tempc)|stddev_samp(temp_tempc)|min(eng_memoryinusemb)|max(eng_memoryinusemb)|avg(eng_memoryinusemb)|stddev_samp(eng_memoryinusemb)|min(eng_cpuuser)|max(eng_cpuuser)|  avg(eng_cpuuser)|stddev_samp(eng_cpuuser)|min(eng_cpubackground)|max(eng_cpubackground)|avg(eng_cpubackground)|stddev_samp(eng_cpubackground)|min(eng_cpusystem)|max(eng_cpusystem)|avg(eng_cpusystem)|stddev_samp(eng_cpusystem)|min(eng_cpuinterrupt)|max(eng_cpuinterrupt)|avg(eng_cpuinterrupt)|stddev_samp(eng_cpuinterrupt)|min(eng_cpuidle)|max(eng_cpuidle)|   avg(eng_cpuidle)|stddev_samp(eng_cpuidle)|
+-----------------+-----------------+-----------------+-------------------+-------------------------+--------------+--------------+------------------+----------------------+-----------------+-----------------+------------------+-------------------------+---------------+---------------+------------------+-----------------------+----------------------+----------------------+----------------------+------------------------------+----------------+----------------+------------------+------------------------+----------------------+----------------------+----------------------+------------------------------+------------------+------------------+------------------+--------------------------+---------------------+---------------------+---------------------+-----------------------------+----------------+----------------+-------------------+------------------------+
|64-64-9B-50-90-00|              0.0|             0.53|0.10169649241477485|      0.07779322052116579|          35.0|          41.0|37.999284180386546|    1.5858533874163507|             35.0|             41.0|37.999284180386546|       1.5858533874163507|           26.0|           50.0|  36.3323448205338|      6.971333929846593|                  1056|                  1056|                1056.0|                           0.0|               4|              90| 6.316392269148174|       7.081149315678373|                     0|                     0|                   0.0|                           0.0|                 2|                 5|2.5876879026485327|        0.5512734308205174|                    0|                    2|  0.14602720114531137|          0.35718284170136083|               6|              94|  90.91696492483894|       7.174665202855025|
|0C-81-26-C6-FF-6C|             1.56|             9.46| 4.8127487845609975|       1.1026322439813494|          45.0|          51.0|48.276270006958946|    1.6412115106167102|             45.0|             51.0|48.276270006958946|       1.6412115106167102|           45.0|           53.0|  49.5661099512874|     2.0192201343616016|                   252|                   272|    266.82254697286015|             8.761834996633704|               8|              37|24.806541405706334|       4.768808805649745|                     0|                     0|                   0.0|                           0.0|                47|                82| 62.74599860821155|         4.316755937653627|                    0|                    8|      3.9053583855254|           1.3949161187584207|               0|              43|  8.563674321503132|       7.663176686126077|
|5C-45-27-A9-65-80|             0.08|             1.15| 0.4238498949182373|      0.16671111455126647|          28.0|          45.0| 36.04308547602502|     6.605680325464992|             28.0|             45.0| 36.04308547602502|        6.605680325464992|           28.0|           46.0| 36.66956219596943|        6.5101980543418|                   252|                   272|    252.90340514246003|             4.153728534077739|               4|              24|10.777623349548298|        4.38799663227597|                     0|                     0|                   0.0|                           0.0|                 2|                24|10.961084086170953|         6.618214374272834|                    0|                    4|   1.5427380125086867|           0.8619895899331574|              51|              93|   76.7560806115358|      11.577461839836511|
|0C-81-26-C6-E9-23|             0.66|             1.81| 1.1293750032782555|       0.3097651235215384|          41.0|          41.0|              41.0|                   0.0|             41.0|             41.0|              41.0|                      0.0|           41.0|           43.0|              42.0|     1.0160010160015247|                   272|                   272|                 272.0|                           0.0|               7|              22|            14.875|      4.5983868139079584|                     0|                     0|                   0.0|                           0.0|                10|                78|              46.5|        25.244609772793975|                    1|                   17|               6.0625|            4.406282172154339|               0|              81|               32.5|      30.638369366194325|
|0C-81-26-C7-06-65|              0.2|             2.21| 0.8652916664154165|       0.2999892070218623|          44.0|          49.0|47.043055555555554|    1.5808269773465842|             44.0|             49.0|47.043055555555554|       1.5808269773465842|           44.0|           53.0| 48.83020833333333|      2.445940123881707|                   272|                   330|    275.07708333333335|            11.194030431335845|               5|              52| 17.64027777777778|       6.893909695658859|                     0|                     0|                   0.0|                           0.0|                 8|                60|              27.6|        10.958179366710402|                    0|                    9|   3.1395833333333334|           1.9914621078116446|               1|              83|  51.63472222222222|       19.22499573976773|
|84-B5-9C-C2-79-E0|              0.0|             1.22|0.15703266134706945|      0.12854608392379258|          51.0|          63.0| 54.66018068102849|    2.0630972868854225|             51.0|             63.0| 54.66018068102849|       2.0630972868854225|           30.0|           63.0| 41.41116516099143|      7.411990227249081|                  1290|                  1290|                1290.0|          1.121086537976429...|               1|              11|1.6497567755385685|      1.4273024889568378|                     0|                     0|                   0.0|                           0.0|                 2|                17| 4.320361362056984|        2.4254608428063738|                    0|                    1|  0.08686587908269632|           0.2816404561143559|              74|              97|  93.87630298818624|      3.7903979895615714|
|0C-81-26-C6-EC-25|             0.25|             2.27| 0.9931988869696565|      0.33939679030943165|          44.0|          48.0| 46.37899860917942|     1.211092430464533|             44.0|             48.0| 46.37899860917942|        1.211092430464533|           44.0|           51.0|  47.8856050069541|     1.9476062495803654|                   252|                   272|    266.32545201668984|              9.01769038033629|               6|              47| 20.53546592489569|       8.572623412086124|                     0|                     0|                   0.0|                           0.0|                 9|                54|27.683588317107095|         9.089967051522386|                    0|                    8|    2.230876216968011|           1.5717836107216872|               3|              83| 49.549374130737135|      18.410620648191895|
|3C-8C-93-94-95-AC|             0.29|              2.6| 0.9610793863041819|      0.34739075896785787|          52.0|          58.0|56.130222841225624|    1.9081391469996736|             52.0|             58.0|56.130222841225624|       1.9081391469996736|           52.0|           60.0| 57.12256267409471|     2.1095772668391235|                   252|                   272|     255.3983286908078|             7.512495384075513|               6|              68|20.761142061281337|       8.746161888063725|                     0|                     0|                   0.0|                           0.0|                 9|                51| 27.56267409470752|         9.801135119024211|                    0|                    9|    2.522284122562674|            1.670171217855932|               3|              84|  49.16991643454039|       19.32198961679761|
|D8-B1-22-88-AF-4F|             0.43|             5.38| 1.8729874683663374|       0.8743071675164259|          43.0|          48.0| 44.76114206128134|    1.7168153808467137|             43.0|             48.0| 44.76114206128134|       1.7168153808467137|           43.0|           51.0|46.306406685236766|     2.3282047023056833|                   252|                   272|    266.37325905292477|             8.994596975174884|               5|              36| 17.98050139275766|       7.059850325410921|                     0|                     0|                   0.0|                           0.0|                 9|                70| 33.57381615598886|        15.027785144803861|                    0|                   13|   3.1399721448467965|           2.3936618966416696|               0|              83| 45.358635097493035|      22.811668217036647|
|64-64-9B-51-12-00|              0.0|             0.48|0.12570200557220076|      0.08764962432638478|          34.0|          40.0| 37.42765042979943|     1.567955411657906|             34.0|             40.0| 37.42765042979943|        1.567955411657906|           27.0|           48.0| 36.18573475235366|      6.386198558284849|                  1056|                  1056|                1056.0|                           0.0|               5|              90| 7.745702005730659|      6.7120022540933455|                     0|                     0|                   0.0|                           0.0|                 2|                 5| 2.555873925501433|        0.5489537351396162|                    0|                    1|  0.08094555873925502|           0.2727654467682738|               6|              93|  89.50716332378224|       6.754147897646813|
|0C-81-26-C7-05-4D|             0.19|             2.57| 0.9204809030220229|      0.33579019675763155|          44.0|          54.0|51.487270155586984|    2.5475000560174617|             44.0|             54.0|51.487270155586984|       2.5475000560174617|           44.0|           57.0| 52.80162659123055|      2.903939758111529|                   252|                   272|     258.9306930693069|             9.519001826343302|               5|              83| 20.31895332390382|       9.206630546577117|                     0|                     0|                   0.0|                           0.0|                 9|                50| 27.26166902404526|         9.833678134524018|                    0|                   10|   2.1725601131541725|            1.525144824843182|               4|              83|  50.26944837340877|        19.4267390215882|
|84-C1-C1-D6-07-E0|max temp      0.0|high memory  1.14| 0.1915194714948359|       0.1325114748056026|          54.0|          77.0| 64.59492350486788|     8.294587540448601|             54.0|             77.0| 64.59492350486788|        8.294587540448601|           33.0|           77.0| 46.59028743625406|      9.524774391146723|                   737|                   942|     821.5476356050069|             84.78783845955115|               4|              36|7.7635605006954105|      3.8335170054849073|                     0|                     0|                   0.0|                           0.0|                 1|                34|   4.8442280945758|         4.689833445398806|                    0|                    3|   0.3198887343532684|           0.5390532102425134|              29|              94|  86.98504867872045|       8.593926226639269|
|C8-E7-F0-FC-72-E7|cpu high     1.38|             3.63|  2.317839998071844|       0.3515347937846861|          27.0|          37.0| 32.13309090909091|     3.430726065105785|             27.0|             37.0| 32.13309090909091|        3.430726065105785|           22.0|           37.0| 30.24981818181818|      4.065090288100114|                   330|                   350|                348.08|              5.89236053813769|              37|              54|45.013090909090906|      2.6092794475166623|                     0|                     0|                   0.0|                           0.0|                28|                55|44.383272727272725|        2.8888628344474685|                    4|                   16|   10.221818181818183|           1.7535655526191687|               0|              20|0.31054545454545457|      1.1563992408129726|
|28-A2-4B-81-0C-97|             0.13|             1.73| 0.8209944352279618|      0.22537247255161416|          26.0|          33.0|29.081363004172463|    1.5349587738754895|             26.0|             33.0|29.081363004172463|       1.5349587738754895|           24.0|           33.0|28.349095966620308|     1.7133785405248394|                   252|                   252|                 252.0|                           0.0|               7|              40|27.091794158553547|       6.428174549945207|                     0|                     0|                   0.0|                           0.0|                 8|                67| 42.16203059805285|        11.544271813297259|                    0|                   10|     4.68567454798331|            2.077932993740253|               2|              84| 26.068150208623088|      19.296356654258975|
|0C-81-26-C7-12-90|heavy pkts    0.3|             2.63| 1.0247284114402317|       0.3391603488860602|          53.0|          56.0|54.801532033426184|    1.0729038613011381|             53.0|             56.0|54.801532033426184|       1.0729038613011381|           53.0|           59.0|56.139275766016716|     1.6955994115768136|                   233|                   252|    251.98676880222843|            0.5013034957732624|               7|              74|20.430362116991642|       7.208785431631524|                     0|                     0|                   0.0|                           0.0|                10|                62| 33.00487465181058|        11.501638181668955|                    0|                    9|   2.9832869080779942|           1.7946860074699627|               2|              80|  43.59540389972145|      19.549327257334884|
|E8-B6-C2-60-10-E0|             1.78|             4.05| 2.7482310476716245|       0.3757257829947299|          28.0|          38.0| 33.24404332129964|     3.024544638575427|             28.0|             38.0| 33.24404332129964|        3.024544638575427|           27.0|           38.0| 32.75848375451263|     3.1455873047050544|                   330|                   350|     346.7509025270758|             7.378017410363712|              39|              55|  47.4028880866426|       2.337661303571087|                     0|                     0|                   0.0|                           0.0|                27|                51| 42.04693140794224|        2.2293078869459895|                    6|                   14|   10.100361010830325|           1.4293369169029853|               0|              13|0.35090252707581226|      0.8444508377836951|
+-----------------+-----------------+-----------------+-------------------+-------------------------+--------------+--------------+------------------+----------------------+-----------------+-----------------+------------------+-------------------------+---------------+---------------+------------------+-----------------------+----------------------+----------------------+----------------------+------------------------------+----------------+----------------+------------------+------------------------+----------------------+----------------------+----------------------+------------------------------+------------------+------------------+------------------+--------------------------+---------------------+---------------------+---------------------+-----------------------------+----------------+----------------+-------------------+------------------------+
* max temp: high memory, didn't see heavy traffic and cpu use, cpu and eng temp high but other temps has normal time
* assumption: memory and cpu/eng temp correlate
* BUT, only 84-C1-C1-D6-07-E0 show good fitting, others have r2~0 due to coef~0
    
dfch1.agg(min_('eng_loadavg1'),max_('eng_loadavg1'),avg('eng_loadavg1'),stddev('eng_loadavg1'), \
                    min_('eng_tempc'),max_('eng_tempc'),avg('eng_tempc'),stddev('eng_tempc'), \
                    min_('eng_cputempc'),max_('eng_cputempc'),avg('eng_cputempc'),stddev('eng_cputempc'), \
                    min_('temp_tempc'),max_('temp_tempc'),avg('temp_tempc'),stddev('temp_tempc'), \
                    min_('eng_memoryinusemb'),max_('eng_memoryinusemb'),avg('eng_memoryinusemb'),stddev('eng_memoryinusemb'), \
                    min_('eng_cpuuser'),max_('eng_cpuuser'),avg('eng_cpuuser'),stddev('eng_cpuuser'), \
                    min_('eng_cpubackground'),max_('eng_cpubackground'),avg('eng_cpubackground'),stddev('eng_cpubackground'), \
                    min_('eng_cpusystem'),max_('eng_cpusystem'),avg('eng_cpusystem'),stddev('eng_cpusystem'), \
                    min_('eng_cpuinterrupt'),max_('eng_cpuinterrupt'),avg('eng_cpuinterrupt'),stddev('eng_cpuinterrupt'), \
                    min_('eng_cpuidle'),max_('eng_cpuidle'),avg('eng_cpuidle'),stddev('eng_cpuidle'))

+-----------------+-----------------+-------------------+-------------------------+--------------+--------------+------------------+----------------------+-----------------+-----------------+------------------+-------------------------+---------------+---------------+----------------+-----------------------+----------------------+----------------------+----------------------+------------------------------+----------------+----------------+-----------------+------------------------+----------------------+----------------------+----------------------+------------------------------+------------------+------------------+------------------+--------------------------+---------------------+---------------------+---------------------+-----------------------------+----------------+----------------+-----------------+------------------------+
|min(eng_loadavg1)|max(eng_loadavg1)|  avg(eng_loadavg1)|stddev_samp(eng_loadavg1)|min(eng_tempc)|max(eng_tempc)|    avg(eng_tempc)|stddev_samp(eng_tempc)|min(eng_cputempc)|max(eng_cputempc)| avg(eng_cputempc)|stddev_samp(eng_cputempc)|min(temp_tempc)|max(temp_tempc)| avg(temp_tempc)|stddev_samp(temp_tempc)|min(eng_memoryinusemb)|max(eng_memoryinusemb)|avg(eng_memoryinusemb)|stddev_samp(eng_memoryinusemb)|min(eng_cpuuser)|max(eng_cpuuser)| avg(eng_cpuuser)|stddev_samp(eng_cpuuser)|min(eng_cpubackground)|max(eng_cpubackground)|avg(eng_cpubackground)|stddev_samp(eng_cpubackground)|min(eng_cpusystem)|max(eng_cpusystem)|avg(eng_cpusystem)|stddev_samp(eng_cpusystem)|min(eng_cpuinterrupt)|max(eng_cpuinterrupt)|avg(eng_cpuinterrupt)|stddev_samp(eng_cpuinterrupt)|min(eng_cpuidle)|max(eng_cpuidle)| avg(eng_cpuidle)|stddev_samp(eng_cpuidle)|
+-----------------+-----------------+-------------------+-------------------------+--------------+--------------+------------------+----------------------+-----------------+-----------------+------------------+-------------------------+---------------+---------------+----------------+-----------------------+----------------------+----------------------+----------------------+------------------------------+----------------+----------------+-----------------+------------------------+----------------------+----------------------+----------------------+------------------------------+------------------+------------------+------------------+--------------------------+---------------------+---------------------+---------------------+-----------------------------+----------------+----------------+-----------------+------------------------+
|              0.0|             9.46|0.25088003883291843|      0.41196638551858694|          26.0|          77.0|62.114900639269216|     10.46067877529708|             26.0|             77.0|62.114900639269216|        10.46067877529708|           22.0|           77.0|45.7073232634533|      9.658956775266416|                   233|                  1290|     832.5845682420667|            194.22454178966674|               1|              90|8.164124679769735|       6.376638917397502|                     0|                     0|                   0.0|                           0.0|                 1|                82| 6.107043149377254|          8.25172734330157|                    0|                   17|   0.5054453389079901|           1.3193893132671715|               0|              97|85.14221383353309|       15.20391944514763|
+-----------------+-----------------+-------------------+-------------------------+--------------+--------------+------------------+----------------------+-----------------+-----------------+------------------+-------------------------+---------------+---------------+----------------+-----------------------+----------------------+----------------------+----------------------+------------------------------+----------------+----------------+-----------------+------------------------+----------------------+----------------------+----------------------+------------------------------+------------------+------------------+------------------+--------------------------+---------------------+---------------------+---------------------+-----------------------------+----------------+----------------+-----------------+------------------------+

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['eng_memoryinusemb'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(dfch.filter(col('id')=='84-C1-C1-D6-07-E0'))
vhouse_df = vhouse_df.select(['features', 'eng_cputempc'])
vhouse_df.show(3)

from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='eng_cputempc', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(vhouse_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

for global fitting, r2 = 0.15, not good
for 84-C1-C1-D6-07-E0, r2 = 0.92, very good. 
Coefficients: [0.09044101112534765]                                             
Intercept: -9.70667534688762
RMSE: 2.337923
r2: 0.920554

In [ ]:
ids = [i['id'] for i in dfch.select(col('id')).distinct().collect()]
for i in ids:
    from pyspark.ml.feature import VectorAssembler
    vectorAssembler = VectorAssembler(inputCols = ['eng_memoryinusemb'], outputCol = 'features')
    vhouse_df = vectorAssembler.transform(dfch.filter(col('id')==i))
    vhouse_df = vhouse_df.select(['features', 'eng_cputempc'])
    print (i)

    from pyspark.ml.regression import LinearRegression
    lr = LinearRegression(featuresCol = 'features', labelCol='eng_cputempc', maxIter=10, regParam=0.3, elasticNetParam=0.8)
    lr_model = lr.fit(vhouse_df)
    print("Coefficients: " + str(lr_model.coefficients))
    print("Intercept: " + str(lr_model.intercept))

    trainingSummary = lr_model.summary
    print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
    print("r2: %f" % trainingSummary.r2)
    
only 84-C1-C1-D6-07-E0 has good fitting, all the others have r2~0, bc coef~0


In [ ]:
dfch1 = dfch.withColumn('eng_loadavg1', dfch.eng_loadavg[0]) \
            .withColumnRenamed('id','id1') \
            .withColumnRenamed('ts','ts1')
dfch1 = dfch1.join(dfin, (dfch1.id1==dfin.id)&(dfch1.ts1==dfin.ts), 'inner')

cols = ['eng_loadavg1','eng_tempc','eng_cputempc','temp_tempc','eng_memoryinusemb','eng_cpuuser','eng_cpubackground','eng_cpusystem','eng_cpuinterrupt','eng_cpuidle', 'rx_pkts','tx_pkts','poe_power']

df_err = dfch1.select(cols).persist(pyspark.StorageLevel.MEMORY_AND_DISK)

vectorAss = VectorAssembler(inputCols=df_err.columns, outputCol="features")
vdf = vectorAss.transform(df_err)
r1 = Correlation.corr(vdf.select('features'), "features").head()
r2 = Correlation.corr(vdf.select('features'), "features")
r2.toPandas().to_csv('corr1.csv')

'eng_loadavg1','eng_tempc','eng_cputempc','temp_tempc','eng_memoryinusemb','eng_cpuuser','eng_cpubackground','eng_cpusystem','eng_cpuinterrupt','eng_cpuidle'

# pkts didn't show strong corr w others
# engtemp high corr cputemp
# eng/cpu temp corr memory, -cpuinterrupt
# memory corr -cpuinterrupt


	pearson(features)
0	"DenseMatrix([[ 1.00000000e+00, -3.31216862e-02, -3.31216862e-02,
              -9.52941771e-02, -2.02874703e-01,  6.49117520e-01,
                          nan,  7.17289175e-01,  6.60029371e-01,
              -7.17200588e-01,  2.73552331e-02,  4.54024539e-02,
               7.71425471e-02],
             [-3.31216862e-02,  1.00000000e+00,  1.00000000e+00,
               1.54245896e-01,  4.98620152e-01, -5.57821547e-02,
                          nan, -2.24638357e-02, -3.74638807e-01,
               6.59582194e-02, -9.34901826e-03, -1.30975511e-02,
              -2.19573940e-02],
             [-3.31216862e-02,  1.00000000e+00,  1.00000000e+00,
               1.54245896e-01,  4.98620152e-01, -5.57821547e-02,
                          nan, -2.24638357e-02, -3.74638807e-01,
               6.59582194e-02, -9.34901826e-03, -1.30975511e-02,
              -2.19573940e-02],
             [-9.52941771e-02,  1.54245896e-01,  1.54245896e-01,
               1.00000000e+00, -8.58655831e-03, -9.04553712e-02,
                          nan, -9.60361220e-02, -1.50982982e-01,
               1.02337301e-01, -6.46963694e-04,  3.17110885e-04,
              -3.78827117e-03],
             [-2.02874703e-01,  4.98620152e-01,  4.98620152e-01,
              -8.58655831e-03,  1.00000000e+00, -3.20349017e-01,
                          nan, -1.77436645e-01, -4.09962452e-01,
               2.66945696e-01, -2.90487389e-02, -4.00141010e-02,
              -5.09526761e-02],
             [ 6.49117520e-01, -5.57821547e-02, -5.57821547e-02,
              -9.04553712e-02, -3.20349017e-01,  1.00000000e+00,
                          nan,  9.06163459e-01,  7.41686728e-01,
              -9.72574579e-01,  2.98808188e-02,  4.45869691e-02,
               6.84061892e-02],
             [            nan,             nan,             nan,
                          nan,             nan,             nan,
               1.00000000e+00,             nan,             nan,
                          nan,             nan,             nan,
                          nan],
             [ 7.17289175e-01, -2.24638357e-02, -2.24638357e-02,
              -9.60361220e-02, -1.77436645e-01,  9.06163459e-01,
                          nan,  1.00000000e+00,  6.62747659e-01,
              -9.75160438e-01,  3.17806336e-02,  4.50270282e-02,
               6.07016610e-02],
             [ 6.60029371e-01, -3.74638807e-01, -3.74638807e-01,
              -1.50982982e-01, -4.09962452e-01,  7.41686728e-01,
                          nan,  6.62747659e-01,  1.00000000e+00,
              -7.51765041e-01,  3.34594276e-02,  5.01457506e-02,
               7.74803256e-02],
             [-7.17200588e-01,  6.59582194e-02,  6.59582194e-02,
               1.02337301e-01,  2.66945696e-01, -9.72574579e-01,
                          nan, -9.75160438e-01, -7.51765041e-01,
               1.00000000e+00, -3.24896419e-02, -4.73675121e-02,
              -6.83978790e-02],
             [ 2.73552331e-02, -9.34901826e-03, -9.34901826e-03,
              -6.46963694e-04, -2.90487389e-02,  2.98808188e-02,
                          nan,  3.17806336e-02,  3.34594276e-02,
              -3.24896419e-02,  1.00000000e+00,  2.40311685e-01,
               1.78091964e-01],
             [ 4.54024539e-02, -1.30975511e-02, -1.30975511e-02,
               3.17110885e-04, -4.00141010e-02,  4.45869691e-02,
                          nan,  4.50270282e-02,  5.01457506e-02,
              -4.73675121e-02,  2.40311685e-01,  1.00000000e+00,
               1.60962484e-01],
             [ 7.71425471e-02, -2.19573940e-02, -2.19573940e-02,
              -3.78827117e-03, -5.09526761e-02,  6.84061892e-02,
                          nan,  6.07016610e-02,  7.74803256e-02,
              -6.83978790e-02,  1.78091964e-01,  1.60962484e-01,
               1.00000000e+00]])"
                
for single switch 84-C1-C1-D6-07-E0

# load corr engtemp, cputemp, memory, user
# pkts didn't show strong corr w others
# engtemp high corr cputemp
# eng/cpu temp corr memory

i.e. load, cpu/eng temp, memory are corr

	pearson(features)
0	"DenseMatrix([[ 1.00000000e+00,  6.41618729e-01,  6.41618729e-01,
              -1.83802936e-02,  6.90811024e-01,  3.99473051e-01,
                          nan,  4.96143424e-01,  2.55815699e-02,
              -4.51956356e-01,  4.64665533e-03,  4.71588792e-04,
               1.14655353e-04],
             [ 6.41618729e-01,  1.00000000e+00,  1.00000000e+00,
               4.02510860e-02,  9.60086219e-01,  2.30813607e-01,
                          nan,  3.85846365e-01, -1.97119848e-01,
              -3.04720388e-01, -7.95285677e-03, -2.25134192e-03,
              -1.42103362e-04],
             [ 6.41618729e-01,  1.00000000e+00,  1.00000000e+00,
               4.02510860e-02,  9.60086219e-01,  2.30813607e-01,
                          nan,  3.85846365e-01, -1.97119848e-01,
              -3.04720388e-01, -7.95285677e-03, -2.25134192e-03,
              -1.42103362e-04],
             [-1.83802936e-02,  4.02510860e-02,  4.02510860e-02,
               1.00000000e+00,  1.04686383e-03, -3.51631581e-02,
                          nan, -2.09547377e-02, -6.73968942e-02,
               3.05407654e-02, -8.76515047e-03, -2.22116682e-03,
              -2.17042501e-04],
             [ 6.90811024e-01,  9.60086219e-01,  9.60086219e-01,
               1.04686383e-03,  1.00000000e+00,  2.82588752e-01,
                          nan,  4.34822409e-01, -1.43719946e-01,
              -3.57099280e-01, -3.17944255e-04, -9.33101924e-05,
               1.14414881e-05],
             [ 3.99473051e-01,  2.30813607e-01,  2.30813607e-01,
              -3.51631581e-02,  2.82588752e-01,  1.00000000e+00,
                          nan,  9.49856264e-01,  4.62622148e-01,
              -9.85459597e-01,  1.27844509e-02,  4.33018745e-03,
               2.06390063e-04],
             [            nan,             nan,             nan,
                          nan,             nan,             nan,
               1.00000000e+00,             nan,             nan,
                          nan,             nan,             nan,
                          nan],
             [ 4.96143424e-01,  3.85846365e-01,  3.85846365e-01,
              -2.09547377e-02,  4.34822409e-01,  9.49856264e-01,
                          nan,  1.00000000e+00,  3.17627321e-01,
              -9.83887149e-01,  7.60460923e-03,  2.37505126e-03,
               1.29671170e-04],
             [ 2.55815699e-02, -1.97119848e-01, -1.97119848e-01,
              -6.73968942e-02, -1.43719946e-01,  4.62622148e-01,
                          nan,  3.17627321e-01,  1.00000000e+00,
              -4.24809020e-01,  2.49834217e-02,  7.63568594e-03,
               3.52058776e-04],
             [-4.51956356e-01, -3.04720388e-01, -3.04720388e-01,
               3.05407654e-02, -3.57099280e-01, -9.85459597e-01,
                          nan, -9.83887149e-01, -4.24809020e-01,
               1.00000000e+00, -1.11281464e-02, -3.57446010e-03,
              -1.81777440e-04],
             [ 4.64665533e-03, -7.95285677e-03, -7.95285677e-03,
              -8.76515047e-03, -3.17944255e-04,  1.27844509e-02,
                          nan,  7.60460923e-03,  2.49834217e-02,
              -1.11281464e-02,  1.00000000e+00,  7.36629726e-01,
               3.27001478e-01],
             [ 4.71588792e-04, -2.25134192e-03, -2.25134192e-03,
              -2.22116682e-03, -9.33101924e-05,  4.33018745e-03,
                          nan,  2.37505126e-03,  7.63568594e-03,
              -3.57446010e-03,  7.36629726e-01,  1.00000000e+00,
               3.33426604e-01],
             [ 1.14655353e-04, -1.42103362e-04, -1.42103362e-04,
              -2.17042501e-04,  1.14414881e-05,  2.06390063e-04,
                          nan,  1.29671170e-04,  3.52058776e-04,
              -1.81777440e-04,  3.27001478e-01,  3.33426604e-01,
               1.00000000e+00]])"

    

In [ ]:
dfch1.agg(min_('eng_loadavg1'),max_('eng_loadavg1'),avg('eng_loadavg1'),stddev('eng_loadavg1'), \
                    min_('eng_tempc'),max_('eng_tempc'),avg('eng_tempc'),stddev('eng_tempc'), \
                    min_('eng_cputempc'),max_('eng_cputempc'),avg('eng_cputempc'),stddev('eng_cputempc'), \
                    min_('temp_tempc'),max_('temp_tempc'),avg('temp_tempc'),stddev('temp_tempc'), \
                    min_('eng_memoryinusemb'),max_('eng_memoryinusemb'),avg('eng_memoryinusemb'),stddev('eng_memoryinusemb'), \
                    min_('eng_cpuuser'),max_('eng_cpuuser'),avg('eng_cpuuser'),stddev('eng_cpuuser'), \
                    min_('eng_cpubackground'),max_('eng_cpubackground'),avg('eng_cpubackground'),stddev('eng_cpubackground'), \
                    min_('eng_cpusystem'),max_('eng_cpusystem'),avg('eng_cpusystem'),stddev('eng_cpusystem'), \
                    min_('eng_cpuinterrupt'),max_('eng_cpuinterrupt'),avg('eng_cpuinterrupt'),stddev('eng_cpuinterrupt'), \
                    min_('eng_cpuidle')

In [ ]:
'''
for c in ['rx_pkts']:
    df0 = dfin.groupby('id','name').agg(min_(c).alias('min'),max_(c).alias('max'),avg(c).alias('avg'),stddev(c).alias('stdv'))
    df0 = df0.withColumn('attribute', lit(c))
'''

In [ ]:
Cpu temperature x
Routing engine temperature x
Temperatures (several loc on chassis) x avg
Memory in use x
Cpu user x
Cpu background x
Cpu system x
Cpu interrupt x
Cpu idle x
Load avg x


In [ ]:

splits = [-float('inf'),0,1,10,100,1000,10000,100000,1000000,10000000,100000000,1000000000,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="rx_pkts", outputCol="bucket") 
bucketedData = bucketizer.transform(dfin) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|511906|
|   2.0|  7972|
|   3.0| 20805|
|   4.0|  8577|
|   5.0| 27711|
|   6.0|  4473|
|   7.0|  1439|
+------+------+
splits = [-float('inf'),0,1,10,100,1000,10000,100000,1000000,10000000,100000000,1000000000,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="tx_pkts", outputCol="bucket") 
bucketedData = bucketizer.transform(dfin) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|468788|
|   2.0|  9764|
|   3.0| 38911|
|   4.0| 22036|
|   5.0| 37350|
|   6.0|  4591|
|   7.0|  1443|
+------+------+
splits = [-float('inf'),0,0.1,0.2,0.5,1,2,5,10,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="poe_power", outputCol="bucket") 
bucketedData = bucketizer.transform(dfin) 
bucketedData.groupby('bucket').count().sort('bucket').show()
|bucket| count|
+------+------+
|   1.0|533620|
|   3.0|     1|
|   4.0|     1|
|   5.0|  1050|
|   6.0|  4522|
|   7.0| 30789|
|   8.0| 12900|
+------+------+
splits = [-float('inf'),0,0.1,0.2,0.5,1,2,5,10,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_loadavg1", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|260033|
|   2.0|274974|
|   3.0|364872|
|   4.0| 41414|
|   5.0| 10030|
|   6.0| 12804|
|   7.0|  1194|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_tempc", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   3.0|  7892|
|   4.0| 32137|
|   5.0| 19170|
|   6.0|438122|
|   7.0|139680|
|   8.0|328320|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_cputempc", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   3.0|  7892|
|   4.0| 32137|
|   5.0| 19170|
|   6.0|438122|
|   7.0|139680|
|   8.0|328320|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="temp_tempc", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   3.0| 12201|
|   4.0|188901|
|   5.0|548814|
|   6.0|137034|
|   7.0| 23651|
|   8.0| 54720|
+------+------+

splits = [-float('inf'),0,500,1000,1500,2000,3000,5000,10000,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_memoryinusemb", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   1.0| 48410|
|   2.0|828288|
|   3.0| 88623|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_cpuuser", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   1.0|769378|
|   2.0|146065|
|   3.0| 33080|
|   4.0|  5484|
|   5.0|  9818|
|   6.0|  1325|
|   7.0|    60|
|   8.0|    39|
|   9.0|    58|
|  10.0|    14|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_cpubackground", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   1.0|965321|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_cpusystem", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   1.0|816501|
|   2.0| 88332|
|   3.0| 32890|
|   4.0|  7464|
|   5.0| 14048|
|   6.0|  3526|
|   7.0|  2442|
|   8.0|   104|
|   9.0|    14|
+------+------+
splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_cpuinterrupt", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   1.0|957971|
|   2.0|  7350|
+------+------+

splits = [-float('inf'),0,10,20,30,40,50,60,70,80,90,100,float('inf')]
bucketizer = Bucketizer(splits=splits, inputCol="eng_cpuidle", outputCol="bucket") 
bucketedData = bucketizer.transform(dfch1) 
bucketedData.groupby('bucket').count().sort('bucket').show()
+------+------+
|bucket| count|
+------+------+
|   1.0| 15191|
|   2.0|  2609|
|   3.0|  3407|
|   4.0|  3815|
|   5.0|  6441|
|   6.0| 21240|
|   7.0| 38911|
|   8.0| 58978|
|   9.0|261564|
|  10.0|553165|
+------+------+


In [ ]:
# Model 3.1: uplink determination
dfin = dfin.filter(col('type')=='phy')
ratioudf = udf(lambda x: x[0]/(x[1]+1), FloatType())
dfin = dfin.withColumn('ratio', ratioudf(struct('rx_pkts', 'tx_pkts')))

# Results shown in excel
# something unexpected


In [ ]:
# Model 3.4: rxpkts/txpkts
splits = [-float('inf'),0,0.1,0.2,0.5,1,2,5,10,float('inf')] 
bucketizer = Bucketizer(splits=splits, inputCol="ratio", outputCol="bucket") 
bucketedData = bucketizer.transform(dfin) 
bucketedData.groupby('bucket').count().sort('bucket').show()                                                                    
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|531876| 0-0.1 majority are 0 for both pkts
|   2.0|   759| 0.1-0.2
|   3.0|  4692|
|   4.0| 28935|
|   5.0|  4834|
|   6.0|  4392|
|   7.0|   315|
|   8.0|  7080|
+------+------+

b = bucketedData.filter(col('tx_pkts')>0) 
b.groupby('bucket').count().sort('bucket').show()
+------+-----+                                                                  
|bucket|count|
+------+-----+
|   1.0|68604|
|   2.0|  759|
|   3.0| 4692|
|   4.0|28935|
|   5.0| 4819|
|   6.0| 1666|
|   7.0|  308|
|   8.0| 4312|
+------+-----+


In [ ]:
splits = [-float('inf'),0,1,10,100,1000,10000,100000,1000000,10000000,100000000,1000000000,float('inf')]


In [9]:
# uplink of interfaces

In [10]: dfin.groupby('id').agg(countDistinct('name')).show()                                                      
+-----------------+--------------------+                                        
|               id|count(DISTINCT name)|
+-----------------+--------------------+
|64-64-9B-50-90-00|                  32|
|0C-81-26-C6-FF-6C|                  33|
|5C-45-27-A9-65-80|                  69|
|0C-81-26-C6-E9-23|                  33|
|0C-81-26-C7-06-65|                  33|
|84-B5-9C-C2-79-E0|                  68|
|0C-81-26-C6-EC-25|                  33|
|3C-8C-93-94-95-AC|                  33|
|D8-B1-22-88-AF-4F|                  33|
|64-64-9B-51-12-00|                  34|
|0C-81-26-C7-05-4D|                  33|
|84-C1-C1-D6-07-E0|                 117|
|C8-E7-F0-FC-72-E7|                  71|
|28-A2-4B-81-0C-97|                  45|
|0C-81-26-C7-12-90|                  33|
|E8-B6-C2-60-10-E0|                  71|
+-----------------+--------------------+

dfin.filter(col('interface_type')=='phy').groupby('id').agg(countDistinct('name')).show() 
+-----------------+--------------------+                                        
|               id|count(DISTINCT name)|
+-----------------+--------------------+
|64-64-9B-50-90-00|                   5|
|0C-81-26-C6-FF-6C|                  12|
|5C-45-27-A9-65-80|                  48|
|0C-81-26-C6-E9-23|                  12|
|0C-81-26-C7-06-65|                  12|
|84-B5-9C-C2-79-E0|                  48|
|0C-81-26-C6-EC-25|                  12|
|3C-8C-93-94-95-AC|                  12|
|D8-B1-22-88-AF-4F|                  12|
|64-64-9B-51-12-00|                   5|
|0C-81-26-C7-05-4D|                  12|
|84-C1-C1-D6-07-E0|                  96|
|C8-E7-F0-FC-72-E7|                  50|
|28-A2-4B-81-0C-97|                  24|
|0C-81-26-C7-12-90|                  12|
|E8-B6-C2-60-10-E0|                  50|
+-----------------+--------------------+

dfinmax = dfin.groupby('id','ts') \
            .agg(max_('rx_pkts').alias('max_rxpkts'), max_('tx_pkts').alias('max_txpkts')) \
            .withColumnRenamed('id','id1') \
            .withColumnRenamed('ts','ts1')
dfin = dfin.join(dfinmax, (dfin.id==dfinmax.id1)&(dfin.ts==dfinmax.ts1), 'left') \
            .drop('id1','ts1')




SyntaxError: invalid syntax (<ipython-input-9-18ca41c9f498>, line 3)

In [ ]:
# data missing check
dfch.groupby('id').agg(countDistinct('ts')).show() 
w0 = Window.partitionBy('id').orderBy('ts')                                                               
df0 = df.select('id','ts').distinct()                                                                     
df0 = df0.withColumn('ts1', lag('ts').over(w0))                                                           
df0 = df0.na.fill(0)                                                                                      
df0 = df0.withColumn('delta', df0.ts - df0.ts1)                                                            
df0.filter(df0.delta>70000000).sort('delta').show(100)  

In [ ]:
#walmart: 
dfch.filter(col('site_id')=='57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39').select('id').distinct().show()        
+-----------------+                                                             
|               id|
+-----------------+
|64-64-9B-50-90-00|
|64-64-9B-51-12-00|
|C8-E7-F0-FC-72-E7|
|E8-B6-C2-60-10-E0|
+-----------------+
#Abhi's
0C-81-26-C7-05-4D

In [ ]:
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats/oc-stats-staging/dt=2019-09-10/*')
rdd = df.rdd

r = rdd.map(lambda x: [(x['id'], x['when'], t['name'], t['rx_bytes'],t['tx_bytes'],t['rx_bps'],t['tx_bps'],t['rx_packets'],t['tx_packets'],t['mbps'], t['mtu'], t['errors'], t['poe']) for t in x['device']['interfaces']])
r1 = r.flatMap(lambda x: [t for t in x])
#excluding errors for now
r2 = r1.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[12]['enabled'],x[12]['status'],x[12]['power_limit'],x[12]['power'],x[12]['priority'],x[12]['class'],x[12]['mode']))
            
df1 = sqlContext.createDataFrame(r2)
tsudf = udf(lambda x: x[0]*1000000000+x[1], LongType())
df1 = df1.withColumn('ts', tsudf(struct(df1['_2']['seconds'],df1['_2']['nanos']))).drop('_2')
df1 = df1.withColumnRenamed('_1','id') \
        .withColumnRenamed('_3','name') \
        .withColumnRenamed('_4','rx_bytes') \
        .withColumnRenamed('_5','tx_bytes') \
        .withColumnRenamed('_6','rx_bps') \
        .withColumnRenamed('_7','tx_bps') \
        .withColumnRenamed('_8','rx_pkts') \
        .withColumnRenamed('_9','tx_pkts') \
        .withColumnRenamed('_10','mbps') \
        .withColumnRenamed('_11','mtu') \
        .withColumnRenamed('_12','poe_enabled') \
        .withColumnRenamed('_13','poe_status') \
        .withColumnRenamed('_14','poe_power_limit') \
        .withColumnRenamed('_15','poe_power') \
        .withColumnRenamed('_16','poe_priority') \
        .withColumnRenamed('_17','poe_class') \
        .withColumnRenamed('_18','poe_mode')
#r1 = rdd.map(lambda x: [(x['id'], x['when'], t['poe_controller']['max_power'], t['']) for t in x['device']])


In [2]:
1. each interface has a configued mbps (id, name), not each id (800-1000), should be max bps
   446 unique interface (57/446 changed mbps in a day but in very small amount 10e-14 though)
   same for mtu (1500 or so)
2. pkts, bytes are incremental
3. bytearray(b'<\x8c\x93\x94\x95\xac') == [3C 8C 93 94 95 AC] ??? id
4. dont know mbps for id
5. mbps is mega bps configured for id (but under interfaces... should change to id level?)
6. 

[]

In [ ]:
w = Window.partitionBy('id','name').orderBy('ts')
df1 = df1.withColumn('prev_rxbytes', lag('rx_bytes').over(w)) \
        .withColumn('prev_txbytes', lag('tx_bytes').over(w)) \
        .withColumn('prev_rxpkts', lag('rx_pkts').over(w)) \
        .withColumn('prev_txpkts', lag('tx_pkts').over(w)) 
df1 = df1.filter(df1.prev_rxpkts.isNotNull())

df1 = df1.withColumn('rxpkts', df1.rx_pkts - df1.prev_rxpkts) \
            .withColumn('txpkts', df1.tx_pkts - df1.prev_txpkts) \
            .withColumn('rxbytes', df1.rx_bytes - df1.prev_rxbytes) \
            .withColumn('txbytes', df1.tx_bytes - df1.prev_txbytes) \
            .drop('prev_rxpkts','prev_txpkts','prev_rxbytes','prev_txbytes','rx_pkts','tx_pkts','rx_bytes','tx_bytes')

df1.write.parquet('s3://mist-data-science-dev/jing/switch_flat_20190910a')

In [ ]:
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="rxpkts", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|545160|
|   2.0| 13594|
|   3.0|   913|
|   4.0| 37937|
|   5.0|  1878|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="txpkts", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|537890|
|   2.0|  2938|
|   3.0|  7942|
|   4.0| 48980|
|   5.0|  1732|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="rxbytes", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|542265|
|   3.0|  1445|
|   4.0|  6785|
|   5.0| 48987|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="txbytes", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|533451|
|   4.0|  6592|
|   5.0| 59439|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="rx_bps", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+ 
|bucket| count|
+------+------+
|   1.0|572735|0-10
|   2.0|   810|
|   3.0|  5103|
|   4.0|  8512|
|   5.0|  8599|
|   6.0|  3225|
|   7.0|   447|
|   8.0|    51|10-100M
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="tx_bps", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|570109|0-10
|   2.0|    85|
|   3.0|  3709|
|   4.0|  4836|
|   5.0| 15945|
|   6.0|  4313|
|   7.0|   435|
|   8.0|    50|10-100M
+------+------+



In [ ]:
df1.groupby(col('id'),col('ts')).agg(sum_('rx_bps').alias('sum_rxbps'),sum_('tx_bps').alias('sum_txbp'),avg('mbps')).show()

max of sum_rxbps: 88M
max of sum_txbps: 91M

In [ ]:
d = dfin.groupby('id','when').agg(sum_('rx_bps'),sum_('tx_bps')) 
splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="sum(rx_bps)", outputCol="bucket")
rxpkts = bucketizer.transform(d)
rxpkts.sort('bucket').groupby('bucket').count().show()

+------+-----+                                                                  
|bucket|count|
+------+-----+
|   1.0| 4303| 
|   4.0|  278|
|   5.0| 5629|
|   6.0| 2382|
|   7.0|  472|
|   8.0|   60| 10-100M
+------+-----+

splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="sum(tx_bps)", outputCol="bucket")
rxpkts = bucketizer.transform(d)
rxpkts.sort('bucket').groupby('bucket').count().show()

+------+-----+                                                                  
|bucket|count|
+------+-----+
|   1.0| 4303|
|   4.0|  145|
|   5.0| 4555|
|   6.0| 3528|
|   7.0|  535|
|   8.0|   58| 10-100M
+------+-----+